In [2]:
import sys
#sys.path.insert(0,'/path/to/pydensecrf/')
from importlib import reload 
#reload(pydensecrf.densecrf)

import pydensecrf
import pydensecrf.densecrf as dcrf
#del sys.modules['dcrf']

from pydensecrf.utils import unary_from_softmax, create_pairwise_bilateral
from PIL import Image
#from PIL import Image

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
import os
import cv2
import random
import tensorflow as tf

from datetime import datetime as dt    
#%matplotlib inline

#plt.rcParams['image.interpolation'] = 'nearest'
#plt.rcParams['image.cmap'] = 'gray'

In [ ]:
from A20220911_UtilidadesVBS import class_deteriorarImagen as CDI

In [ ]:
import importlib_metadata as ilm

In [6]:
class class_CRF_OutputDatoseImagen():
    

    def __call__(self, imagenUsar, 
                 nombrearchivoIn, 
                 MetodoRGBslots, 
                 elegirRGBslots,
                 influirSobelEnSmothnessKernel, 
                 ParametroXY, 
                 ParametroRGB, 
                 probabilidades, 
                 umbralFiltro, 
                 metodoConv, 
                 k_conv, 
                 metodoFranjas, 
                 nfranjasRow, 
                 nfranjasCol, 
                 grosorFranja, 
                 metodoErosionar, 
                 kernelErosionar, 
                 int_erosionIteraciones,
                 inference,
                 pairwisebilateralCompat,
                 metodoDilatar,
                 probmapTipo,
                 DCNNProbmap,
                 fecha,
                 etiquetaInPath, 
                 rawInPath,
                 numClasesIn):   
        
        
        #print("elegirRGBslots :",elegirRGBslots)
        self.imagenUsar=imagenUsar
        self.elegirRGBslots=elegirRGBslots
        self.umbralFiltro=umbralFiltro
        self.ParametroXY=ParametroXY
        self.ParametroRGB=ParametroRGB        
        self.MetodoRGBslots=MetodoRGBslots
        self.probabilidades=probabilidades
        self.influirSobelEnSmothnessKernel=influirSobelEnSmothnessKernel
        #self.verbose=verbose        
        #self.libreria=(pydensecrf.__file__).rsplit('/', 4)[-4]
        self.nombrearchivoIn=nombrearchivoIn
        self.metodoConv=metodoConv
        self.k_conv=k_conv
        self.metodoFranjas=metodoFranjas
        self.nfranjasRow=nfranjasRow
        self.nfranjasCol=nfranjasCol
        self.grosorFranja=grosorFranja
        #self.prefijo=str(imagenUsar)+str(elegirRGBslots)+"Umb"+str(umbralFiltro)+"kConv"+str(k_conv)+"NFR"+str(nfranjasRow)+"NFC"+str(nfranjasCol)+"GF"+str(grosorFranja)
        
        self.calcularArctanYSobels=None
        self.metodoErosionar=metodoErosionar
        self.kernelErosionar=kernelErosionar
        self.int_erosionIteraciones=int_erosionIteraciones
        self.inference=inference
        self.pairwisebilateralCompat=pairwisebilateralCompat #10
        
        self.metodoDilatar=metodoDilatar
        
        self.probmapTipo=probmapTipo
        self.DCNNProbmap=DCNNProbmap
        self.etiquetaInPath=etiquetaInPath
        self.rawInPath=rawInPath
        self.numClasesIn=numClasesIn
        
        #self.borrame=3 #El probmap tiene que venir del deeplab
        
        self.prefijo=str(imagenUsar)+str(elegirRGBslots)+"Umb"+str(umbralFiltro)+"kConv"+str(k_conv)+"NFR"+str(nfranjasRow)+"NFC"+str(nfranjasCol)+"GF"+str(grosorFranja)+"EroK"+str(kernelErosionar)+"EroIt"+str(int_erosionIteraciones)+"Inf"+str(inference)+"Compat"+str(pairwisebilateralCompat)
        
        
        
        if (r"Víctor" in (ilm.metadata('pydensecrf')["Summary"])):
            self.libreria="Propuesta"
        
        #if("Original" in (pydensecrf.__file__)):
        #    self.libreria="Original"
        else:
            self.libreria="Original"
        
        
        str_fecha=fecha#dt.today().strftime('%Y%m%d_%H%M')
        self.directorioOutputs=r"outputsMasivos/ImagenesTemporales/"+str_fecha+self.prefijo+r"/"
        
        
        if(os.path.exists(self.directorioOutputs)==False):
            os.makedirs(self.directorioOutputs)
        
        
        self.func_validarParametros(self.libreria, self.MetodoRGBslots, self.elegirRGBslots, self.influirSobelEnSmothnessKernel, self.metodoFranjas, self.nfranjasRow, self.nfranjasCol, self.grosorFranja)
        
        self.calcularArctanYSobels=self.func_requiereArctanYSobels(self.elegirRGBslots)
        self.calcularArctanYSobels=True
        
        #self.img_rawIM, self.img_rawIMGris, self.labelG, self.labelG_GT1 = self.func_load_Images(self.imagenUsar)#labelG_GT1 es porque es el label sin deteriorar. labelG_GT va a deteriorarse en la siguiente linea de este script de esta version....
        self.img_rawIM, self.img_rawIMGris, self.labelG, self.labelG_GT1 = self.func_load_Images(self.imagenUsar, 
                                                                                                 self.etiquetaInPath, 
                                                                                                 self.rawInPath)#labelG_GT1 es porque es el label sin deteriorar. labelG_GT va a deteriorarse en la siguiente linea de este script de esta version....
        
        
        ###BLOQUE DETERIORAR INICIO
        #EN esta parte se trata de deteriorar la imagen 
        #print("Class A20220911_CRFVBS: Bloque deteriorar inicio")
        objeto_class_deteriorarImagen=CDI()        
        objeto_class_deteriorarImagen(self.prefijo, self.directorioOutputs)
        
        if (metodoDilatar==True):
            #self.labelG = self.func_deteriorarImagen_Dilatar(self.labelG)
            #a=class_deteriorarImagen()
            #self.labelG = class_deteriorarImagen().func_deteriorarImagen_Dilatar1(self.prefijo, self.directorioOutputs,self.labelG)
            self.labelG = objeto_class_deteriorarImagen.func_deteriorarImagen_Dilatar(self.labelG)
        
        #self.labelG[self.labelG == 1] = 255
        if (metodoErosionar==True):
            #self.img_rawIM, self.img_rawIMGris, self.labelG = self.func_deteriorarImagen_Erosionar(kernelErosionarIn=self.kernelErosionar,kernelErosionarIteracionesIn=self.int_erosionIteraciones,img_rawIM=self.img_rawIM, img_rawIMGris=self.img_rawIMGris, labelG=self.labelG)
            self.img_rawIM, self.img_rawIMGris, self.labelG = objeto_class_deteriorarImagen.func_deteriorarImagen_Erosionar(kernelErosionarIn=self.kernelErosionar,kernelErosionarIteracionesIn=self.int_erosionIteraciones,img_rawIM=self.img_rawIM, img_rawIMGris=self.img_rawIMGris, labelG=self.labelG)
        if (metodoFranjas==True):
            #self.img_rawIM, self.img_rawIMGris, self.labelG, self.labelG_GT = self.func_deteriorarImagen_Grilla(self.img_rawIM, self.img_rawIMGris, self.labelG, self.labelG_GT1)
            #self.img_rawIM, self.img_rawIMGris, self.labelG = self.func_deteriorarImagen_Grilla(self.nfranjasRow, self.nfranjasCol, self.grosorFranja, self.img_rawIM, self.img_rawIMGris, self.labelG)
            self.img_rawIM, self.img_rawIMGris, self.labelG = objeto_class_deteriorarImagen.func_deteriorarImagen_Grilla(self.nfranjasRow, self.nfranjasCol, self.grosorFranja, self.img_rawIM, self.img_rawIMGris, self.labelG)
        if (metodoConv==True):
            #self.img_rawIM, self.img_rawIMGris, self.labelG = self.func_deteriorarImagen_Convolucion(self.k_conv, img_rawIM=self.img_rawIM, img_rawIMGris=self.img_rawIMGris, labelG=self.labelG)#Con kconv=1 se anula esto
            self.img_rawIM, self.img_rawIMGris, self.labelG = objeto_class_deteriorarImagen.func_deteriorarImagen_Convolucion(self.k_conv, img_rawIM=self.img_rawIM, img_rawIMGris=self.img_rawIMGris, labelG=self.labelG)#Con kconv=1 se anula esto
        
        #self.labelG, self.labelGRows, self.labelGCols, self.img_raw, self.img_rawIM, self.img_rawIMGris=self.func_get_NameArrimageArraylabeledRowsCols(self.imagenUsar, self.verbose)
        #self.arrProbmap, self.labelG, self.img_rawIMGris =self.func_get_arrProbmap(self.imagenUsar, self.labelG, self.labelGRows, self.labelGCols, self.probabilidades, self.k_conv, self.metodoFranjas, self.nfranjasRow, self.nfranjasCol, self.grosorFranja)
        
        ###BLOQUE DETERIORAR FIN
        
        #print("Class A20220911_CRFVBS: Generar Probmap")
        if (self.probmapTipo=="probmapgeneradoOriginal"):
            self.arrProbmap = self.func_get_arrProbmap(self.labelG, self.probabilidades)#Aquí no se crean imagenes
            #print(type(self.arrProbmap))
            #print(self.arrProbmap.shape)
            #input("Press a key to continue...")
            
        if (self.probmapTipo=="probmapgeneradoAlternativo"):
            self.arrProbmap = self.func_get_arrProbmap10(self.labelG, self.probabilidades)#Aquí no se crean imagenes
        
        if (self.probmapTipo=="probmapDesdeDCNN"):
            self.arrProbmap=self.DCNNProbmap
            
            
        #print("self.arrProbmap.shape",self.arrProbmap.shape)
        #print(type(self.arrProbmap))
        #(self, imagenUsar, img_rawIM,img_rawIMGris, labelG, k_conv, nfranjasRowIn, nfranjasColIn, grosorFranjaIn):
        
        #self.func_plt_arrProbmap(self.arrProbmap)
        
        #if(MetodoRGBslots=='Nuevo'):
        #if((self.MetodoRGBslots=='MetodoRGBslots_nuevo') or (self.influirSobelEnSmothnessKernel==True) or (self.libreria=='20220211_lucasbeyer_pydensecrf')):#Independiente de la libreria porque puede ser la original pero poner los sobels en los canales, los que arman el canal angulo que se puede usar. Pero tiene que estar la libreria nueva o mas adelante en la funcion en que anula los sobels, se cae porque no estan defnidios los sobelx y sobely
        #if((self.MetodoRGBslots=='MetodoRGBslots_nuevo') or (self.libreria=='20220211_lucasbeyer_pydensecrf')):#La condicion influirSobelEnSmothnessKerneles==True puede ser posible solo con la libreria nueva, y con la libreria nueva siempre se calculan sobels, entonces es redundante poner esa condicion
        
        #print("Class A20220911_CRFVBS: Calcular Arctan y Sobel")
        if((self.calcularArctanYSobels==True) or (self.libreria=='20220211_lucasbeyer_pydensecrf')):#La condicion influirSobelEnSmothnessKerneles==True puede ser posible solo con la libreria nueva, y con la libreria nueva siempre se calculan sobels, entonces es redundante poner esa condicion
            #self.sobelsdx, self.sobelsdy, self.sobelh=self.func_get_sobels(self.img_rawIMGris)
            self.sobelsdx, self.sobelsdy, self.sobelh=self.func_get_sobels(self.img_rawIMGris)
            
        #self.func_plt_sobels(self.sobelsdx, self.sobelsdy, self.sobelh)
        #print("ocatarineta")
           
        #if((self.MetodoRGBslots=='MetodoRGBslots_nuevo')):#Independiente de la libreria porque puede ser la original pero poner los sobels en los canales. Problema: podria ser metodo nuevo pero usar C1ImgGrisC2ImGrisC30 y no necesariamente usar sobels... entonces calcularlos aquí estaría de mas
        if((self.calcularArctanYSobels==True)):            
            self.arctan=self.func_get_Arctan(self.sobelsdx, self.sobelsdy)
            
            if (self.umbralFiltro!="SinUmbral"):
                self.arctan=self.func_filtrar_Umbral(self.arctan, self.sobelh, self.umbralFiltro)
         
        #print("Class A20220911_CRFVBS: Armar imagen propuesta RGB")
        if((self.MetodoRGBslots=='MetodoRGBslots_nuevo')):
            self.img_RGBasignacion, self.img_RGBasignacionArr = self.func_construir_ImagenRGBslots(self.img_rawIM, self.img_rawIMGris, self.elegirRGBslots, self.arctan)#Aca ya se uso sobelx, sobely y se guardo una imagen, ahí esta la info, entonces se puede usar esas mismas variables de nuevo
                
        
        #if((self.libreria=='20220211_lucasbeyer_pydensecrf'==True) & (self.influirSobelEnSmothnessKernel==True)):#Solo tiene sentido si se usa la libreria nueva, con la original da lo mismo esto total no se incluyen sobesl en el parametro d.bilateral
        #if((self.libreria=='20220211_lucasbeyer_pydensecrf')):#Solo tiene sentido si se usa la libreria nueva, con la original da lo mismo esto total no se incluyen sobesl en el parametro d.bilateral
            
            #self.sobelsdx_post, self.sobelsdy_pre, self.sobelh_pre = self.sobelsdx, self.sobelsdy, self.sobelh
        
        
        
        #import cv2
        ########################
        #Dependiendo del metodo del MetodoRGBslots se carga una imagen o la otra
        if(MetodoRGBslots=='MetodoRGBslots_nuevo'):
            self.imgdef=self.img_RGBasignacion
        if(MetodoRGBslots=='MetodoRGBslots_original'):#Aca es la unica parte en donde se usa MetodoRGBslots_original. Viene desde self.arrProbmap y salta hasta aqui (si es que es la libreria original) y viene desde el primer calculo de sobels (si es la nueva libreria)
            
            #img_raw2=cv2.imread(img_raw)
            
            #self.imgdef=cv2.imread(self.img_raw)
            self.imgdef=self.img_rawIM
            
            #if(self.metodoFranjas==True): #Siempre va a estar incluida la convolucin en esta imagen OJO (junto con las franjas)
                #self.imgdef=cv2.imread(self.imagenUsar+"RAW_Franjas.png")
                #self.imgdef=cv2.imread(r"outputsMasivos/RAW_Franjas.png")
             #   self.imgdef=cv2.imread(self.filenameRaw_conv)#Antes tenia un nombre fijo.. ahora depende de los parametros
                
        #MetodoRGBslots=[""]
        
        #self.borrame=cv2.imread(self.img_raw)
        ########################
        
        #print("Class A20220911_CRFVBS: Aplicar CRF (esto es lo mas lento.. quizsa por las dimensiones de la imagen mayores a 1000x1000). Quizas es el numero de inferencias del CRF")
        if(self.libreria=="Original"):
            print("Libreria original")
            self.Q=self.func_aplicar_CRF_LibOriginal(self.arrProbmap, self.ParametroXY, self.ParametroRGB, self.imgdef, self.inference, self.pairwisebilateralCompat)#El bilateral necesita el arrprobmap y la imagen raw
            

        if(self.libreria=="Propuesta"):
            print("Libreria propuesta")
            self.sobelsdx, self.sobelsdy=self.func_influir_SobelEnSmothnessKernel(self.sobelsdx, self.sobelsdy, self.influirSobelEnSmothnessKernel)#Aca intervengo para cambiar valor de los sobels si es que no quiero que influyan en la libreria nueva al obtener self.img_RGBasignacion
            self.Q=self.func_aplicar_CRF_LibNueva(self.sobelsdx, self.sobelsdy, self.arrProbmap, self.ParametroXY, self.ParametroRGB, self.influirSobelEnSmothnessKernel, self.imgdef, self.inference, self.pairwisebilateralCompat)#El bilateral necesita el arrprobmap y la imagen raw
            
        #if((self.libreria=='20220211_lucasbeyer_pydensecrf') & (self.MetodoRGBslots=='Nuevo')):
        #    self.Q=self.func_aplicar_CRF_LibNueva(self.sobelsdx, self.sobelsdy, self.img_RGBasignacion, self.arrProbmap, self.ParametroXY, self.ParametroRGB)
            
        #if(self.libreria=='20220211_lucasbeyer_pydensecrf'):
        #    self.Q=self.func_aplicar_CRF(self.sobelsdx, self.sobelsdy, self.img_raw, self.img_RGBasignacion, self.arrProbmap, self.ParametroXY, self.ParametroRGB, self.MetodoRGBslots, self.libreria)
        ########################

        
        #print("Class A20220911_CRFVBS: Post procesamiento predicciones")
        self.Q=self.func_modificar_outputCRF(self.Q)
        
        #self.num_classesIn=2
        self.miou=self.func_get_mIoU(self.labelG_GT1, self.Q, self.imagenUsar,self.numClasesIn)
        #self.func_plotearguardar_imagen_CRF(self.Q,self.miou,self.elegirRGBslots, self.umbralFiltro, self.ParametroXY,  self.ParametroRGB, self.MetodoRGBslots, self.probabilidades)
        print("self.miou: ", self.miou)
        
        self.func_write_ToCSV(self.miou,self.libreria, self.imagenUsar,self.elegirRGBslots, self.umbralFiltro, self.ParametroXY,  self.ParametroRGB, self.MetodoRGBslots, self.probabilidades, 
                              self.influirSobelEnSmothnessKernel, self.nombrearchivoIn, self.metodoConv,self.k_conv, self.metodoFranjas, self.nfranjasRow, self.nfranjasCol, self.grosorFranja,
                             self.metodoErosionar, self.kernelErosionar, self.int_erosionIteraciones, self.inference, self.pairwisebilateralCompat)
        
        #(self,miou, imagenUsar, elegirRGBslots, umbralFiltro, ParametroXY, ParametroRGB, MetodoRGBslots, probabilidades,influirSobelEnSmothnessKernel, nombrearchivoIn, k_conv, metodoFranjas, nfranjasRow, nfranjasCol, self.grosorFranja):
        
        #self.nombreArchivo=nombreArchivo1    
        #self.list_clientesActuales=clientesActuales1   
        #self.list_subsidiariasActuales=subsidiariasActuales1       

In [7]:
class class_CRF_OutputDatoseImagen(class_CRF_OutputDatoseImagen):   
    
    def func_validarParametros(self, libreria, MetodoRGBslots, elegirRGBslots, influirSobelEnSmothnessKernel, metodoFranjas, nfranjasRow, nfranjasCol, grosorFranja):

        if((MetodoRGBslots=='MetodoRGBslots_original') & (elegirRGBslots!="elegirRGBslots_original")):#Si se usa "MetodoRGBslots_original" entonces elegir "elegirRGBslots_original"-
            raise ValueError("Si usas metodo original, usa elegirRGBslots=[\"elegirRGBslots_original\"]. Si usas metodo original, da lo mismo como administrar los canales poniendoles diferentes imagenes, ya que mas adelante en el codigo, solo se cargara la imagen raw del directorio local para pasarsela como parametro al pairwisebilateral. Ojo que este error recibe un string no una lista, revisa que no sea eso")

        if((libreria=='Original') & (influirSobelEnSmothnessKernel==True)):
            raise ValueError("Si usas libreria original, no es posible incluir sobels en d.bilateral no esta construida esa funcion para admitir esos parametros")

        if((MetodoRGBslots=='MetodoRGBslots_nuevo') & (elegirRGBslots=="elegirRGBslots_original")):
            raise ValueError("Si pones elegirRGBslots=='elegirRGBslots_original', entonces usa MetodoRGBslots=='Original'")

        if((libreria=='Propuesta') & (influirSobelEnSmothnessKernel==False)):
            warnings.warn("Si usas la libreria propuesta, entonces poner influirSobelEnSmothnessKernel==False es algo raro")

        if((metodoFranjas==False) & (nfranjasRow!=0 or nfranjasCol!=0 or grosorFranja!=0)):
            raise ValueError("Si usas la metodoFranjas False, poner nfranjasRow, nfranjasCol, grosorFranja iguales a 0")
            
        return(True)
        

In [8]:
class class_CRF_OutputDatoseImagen(class_CRF_OutputDatoseImagen):   
    
    def func_requiereArctanYSobels(self, elegirRGBslots):
        #bulean=None
        if ("Angulo" in elegirRGBslots):
            bulean=True
        else:
            bulean=False
        return(bulean)          

In [9]:
class class_CRF_OutputDatoseImagen(class_CRF_OutputDatoseImagen):   
    
    def func_load_Images(self, imagenUsar, etiquetaInPath, rawInPath):
        p=0
        if (imagenUsar=="custom"):
            p=p+1
            img_rawIM=np.array(Image.open(rawInPath))#imagen raw
            img_rawIMGris = np.array(Image.open(rawInPath).convert('L'))#imagen raw gris
            
            labelG=np.array(Image.open(etiquetaInPath).convert('L'))
            labelG[labelG == 1] = 255#Asi tendremos solo pixeles 0 y 255. Se normaliza entre 0 y 1 mas adelante
            labelG_GT=np.array(Image.open(etiquetaInPath).convert('L'))
        
            
        
        if (imagenUsar=="usarEmbrion2Clases"):
            p=p+1
            
            #img_rawIM=np.array(Image.open("Raw0.jpg"))
            #img_rawIM=np.array(Image.open("Raw0.jpg"))#imagen raw
            #img_rawIMGris = np.array(Image.open("Raw0.jpg").convert('L'))#imagen raw gris
            
            #img_rawIM=np.array(Image.open(r"./COLAB_instance-level-human-parsing2/embrion/secuencia/raw/embryoR-10000.jpg"))#imagen raw
            #img_rawIMGris = np.array(Image.open(r"./COLAB_instance-level-human-parsing2/embrion/secuencia/raw/embryoR-10000.jpg").convert('L'))#imagen raw gris
            
            #pathMio=r"./COLAB_instance-level-human-parsing2/embryoDataset/video_animal2_proyeccion_SeqDA/1OR0000.jpg"#
            img_rawIM=np.array(Image.open(r"./COLAB_instance-level-human-parsing2/embryoDataset/video_animal2_proyeccion_SeqDA/1OR0000.jpg"))#imagen raw
            img_rawIMGris = np.array(Image.open(r"./COLAB_instance-level-human-parsing2/embryoDataset/video_animal2_proyeccion_SeqDA/1OR0000.jpg").convert('L'))#imagen raw gris
            
            
            
            #labelG=np.array(Image.open("combinadasout0_2clases.png").convert('L'))
            labelG=np.array(Image.open(r"./COLAB_instance-level-human-parsing2/embryoDataset/EVLstack_SeqDA/1OR0000.png").convert('L'))
            labelG[labelG == 1] = 255#Asi tendremos solo pixeles 0 y 255. Se normaliza entre 0 y 1 mas adelante
            #labelG_GT=np.array(Image.open("combinadasout0_2clases.png").convert('L'))
            labelG_GT=np.array(Image.open(r"./COLAB_instance-level-human-parsing2/embryoDataset/EVLstack_SeqDA/1OR0000.png").convert('L'))
        
        if (imagenUsar=="usarMSRC"):
            p=p+1
            img_rawIM=np.array(Image.open("1_13_s.bmp"))#imagen raw
            img_rawIMGris = np.array(Image.open("1_13_s.bmp").convert('L'))#imagen raw gris
            labelG=np.array(Image.open("20220526_MRSCVaca.png").convert('L'))
            labelG[labelG >= 1] = 255#Asi tendremos solo pixeles 0 y 255. Se normaliza entre 0 y 1 mas adelante
            labelG_GT=np.array(Image.open("20220526_MRSCVaca.png").convert('L'))
            labelG_GT[labelG_GT >= 1] = 1#No es necesario porque se supone que el label ya es con 0 para fondo y 1 para vaca (pero parece que en realidad es 255 para vaca)
            
            
        img_rawIM_out="0rawIMLoaded_"+self.prefijo+".png" #8 entre parentesis porque es como la imagen "8ImagenRGBslots", del metodo original
        img_rawIMGris_out="0rawIMGrisLoaded_"+self.prefijo+".png"
        labelG_out="0labelGLoaded_"+self.prefijo+".png"
        labelG_GT_out="0labelG_GTLoaded_"+self.prefijo+".png"

        #filenameGroundTruth=r"outputsMasivos/ImagenesTemporales/Embrion2C_GT.png"                        
        #if(os.path.exists(filenameGroundTruth)==False):
        #    img_aux = Image.fromarray(labelG_GT)
        #    img_aux.save(filenameGroundTruth)

        if(os.path.exists(self.directorioOutputs+img_rawIM_out)==False):#guardar imagen convolucionada una sola vez
                img_aux2 = Image.fromarray(img_rawIM)
                img_aux2.save(self.directorioOutputs+img_rawIM_out)

        if(os.path.exists(self.directorioOutputs+img_rawIMGris_out)==False):#guardar imagen convolucionada una sola vez
                img_aux2 = Image.fromarray(img_rawIMGris)
                img_aux2.save(self.directorioOutputs+img_rawIMGris_out)

        if(os.path.exists(self.directorioOutputs+labelG_out)==False):#guardar imagen convolucionada una sola vez
                img_aux2 = Image.fromarray(labelG)
                img_aux2.save(self.directorioOutputs+labelG_out)

        if(os.path.exists(self.directorioOutputs+labelG_GT_out)==False):#guardar imagen convolucionada una sola vez
                img_aux2 = Image.fromarray(labelG_GT)
                img_aux2.save(self.directorioOutputs+labelG_GT_out)


            
        if (p!=1):
            raise ValueError("No cargo ninguna imagen o cargó mas de una imagen con en lnombre de imagen ingresado")  
        return(img_rawIM, img_rawIMGris, labelG, labelG_GT)

In [10]:
#np.ones((3,3), np.uint8)*2

In [11]:
if False:
    class class_CRF_OutputDatoseImagen(class_CRF_OutputDatoseImagen):   

        def func_deteriorarImagen_Dilatar(self, labelG,*args,**kwargs):#Deja a todos del mismo color, no en degrade como con la convolucion entonces es mas dificil para el crf 

            dilating_kernel = np.ones((5, 5), 'uint8')
            labelG = cv2.dilate(labelG, dilating_kernel, iterations=3)

            nombreImagenLabeled_dilated="1labelDilated_"+self.prefijo+".png"                        

            if(os.path.exists(self.directorioOutputs+nombreImagenLabeled_dilated)==False):#guardar imagen convolucionada una sola vez
                    img_aux2 = Image.fromarray(labelG)
                    img_aux2.save(self.directorioOutputs+nombreImagenLabeled_dilated)

            return(labelG)

In [12]:
if False:
    class class_CRF_OutputDatoseImagen(class_CRF_OutputDatoseImagen):  

        #@classmethod
        def func_deteriorarImagen_Erosionar(self, kernelErosionarIn,kernelErosionarIteracionesIn,*args, **kwargs):
        #def func_deteriorarImagen_Erosionar(self, img_rawIM,img_rawIMGris, labelG):    


            img_rawIM, img_rawIMGris, labelG = kwargs.get("img_rawIM"), kwargs.get("img_rawIMGris"), kwargs.get("labelG")
            #imagenUsar, k_conv, nfranjasRowIn, nfranjasColIn, grosorFranjaIn = self.imagenUsar, self.k_conv, self.nfranjasRow, self.nfranjasCol, self.grosorFranja

            #print(r"kwargs.get('asdasdsa')",kwargs.get("asdasdsa"))
            #int_erosionIteraciones=1o{}
            #img_rawIM2=img_rawIM
            #img_rawIMGris2=img_rawIMGris
            #labelG2=labelG

            #img_rawIM2[img_rawIM2 == 255] = 255//2
            #img_rawIMGris2[img_rawIMGris2 == 255] = 255//2
            #labelG2[labelG2 == 255] = 255//2
            lst_output=[]

            eroding_kernel = np.ones((kernelErosionarIn,kernelErosionarIn), np.uint8)

            if (img_rawIM is not None):
            #ImagenRGB
                img_rawIM[:,:,0] = cv2.erode(img_rawIM[:,:,0], eroding_kernel, iterations = kernelErosionarIteracionesIn)#Esta es para calcular kernel bilateral
                img_rawIM[:,:,1] = cv2.erode(img_rawIM[:,:,1], eroding_kernel, iterations = kernelErosionarIteracionesIn)#Esta es para calcular kernel bilateral
                img_rawIM[:,:,2] = cv2.erode(img_rawIM[:,:,2], eroding_kernel, iterations = kernelErosionarIteracionesIn)#Esta es para calcular kernel bilateral

                nombreImagenRaw_erode="1rawIMerode_"+self.prefijo+".png" #8 entre parentesis porque es como la imagen "8ImagenRGBslots", del metodo original

                if(os.path.exists(self.directorioOutputs+nombreImagenRaw_erode)==False):#guardar imagen convolucionada una sola vez
                    img_aux2 = Image.fromarray(img_rawIM)
                    img_aux2.save(self.directorioOutputs+nombreImagenRaw_erode)

                lst_output.append(img_rawIM)
                #print("Entro a img_rawIM")

            if (img_rawIMGris is not None):
            #ImagenGris
                img_rawIMGris = cv2.erode(img_rawIMGris, eroding_kernel, iterations = self.int_erosionIteraciones)#Esta es para calcular kernel bilateral

                nombreImagenRawGris_erode="1rawIMGriserode_"+self.prefijo+".png"

            #ImagenEtiquetada
            #labelG2=labelG
            #labelG[labelG == 255] = 255//2
            #labelGAux = cv2.erode(labelG2, eroding_kernel, iterations = self.int_erosionIteraciones)#Esta es para calcular probmap
            #labelG[labelGAux>0] = labelGAux[labelGAux>0]
                if(os.path.exists(self.directorioOutputs+nombreImagenRawGris_erode)==False):#guardar imagen convolucionada una sola vez
                    img_aux2 = Image.fromarray(img_rawIMGris)
                    img_aux2.save(self.directorioOutputs+nombreImagenRawGris_erode)
                lst_output.append(img_rawIMGris)
                #print("Entro a img_rawIMGris")


            if(labelG is not None):
                labelG = cv2.erode(labelG, eroding_kernel, iterations = self.int_erosionIteraciones)#Esta es para calcular probmap

                nombreImagenLabeled_erode="1labelGerode_"+self.prefijo+".png" 
                if(os.path.exists(self.directorioOutputs+nombreImagenLabeled_erode)==False):#guardar imagen convolucionada una sola vez
                    img_aux2 = Image.fromarray(labelG)
                    img_aux2.save(self.directorioOutputs+nombreImagenLabeled_erode)
                lst_output.append(labelG)
                #print("Entro a labelG")

            #if (imagenUsar=="usarEmbrion2Clases"):
            #Estas imagenes pueden estar o no convolucionadas (con k_conv=1 quedarian igual), y pueden o no tener franjas





            #nombreImagenRaw_conv=r"outputsMasivos/ImagenesTemporales/"+imagenUsar+"_RawRGBkconv"+str(k_conv)+"franjas"+str(nfranjasRowIn)+str(nfranjasColIn)+str(grosorFranjaIn)+".png"#las convolucionadas pueden tener o no franjas        
            #nombreImagenRawGris_conv=r"outputsMasivos/ImagenesTemporales/"+imagenUsar+"_RawGriskconv"+str(k_conv)+"franjas"+str(nfranjasRowIn)+str(nfranjasColIn)+str(grosorFranjaIn)+".png"#las convolucionadas pueden tener o no franjas        
            #nombreImagenLabeled_conv=r"outputsMasivos/ImagenesTemporales/"+imagenUsar+"_Labkconv"+str(k_conv)+"franjas"+str(nfranjasRowIn)+str(nfranjasColIn)+str(grosorFranjaIn)+".png"

            #self.filenameRaw_conv=filenameRaw_conv#para poder tirar esta linea self.imgdef=cv2.imread(self.filenameRaw)

            #imagenConvolucionada=imagenUsar+"Conv_Franjas.png"
            #print("lst_output",lst_output)

            return(lst_output)

            if False:
                print("as")
                #Parameters
                #src	input image; the number of channels can be arbitrary, but the depth should be one of cv.CV_8U, cv.CV_16U, cv.CV_16S, cv.CV_32F or cv.CV_64F.
                #dst	output image of the same size and type as src.
                #kernel	structuring element used for erosion.
                #anchor	position of the anchor within the element; default value new cv.Point(-1, -1) means that the anchor is at the element center.
                #iterations	number of times erosion is applied.
                #borderType	pixel extrapolation method(see cv.BorderTypes).
                #borderValue	border value in case of a constant border

In [13]:
if False:
    class class_CRF_OutputDatoseImagenDESUSO(class_CRF_OutputDatoseImagen):   

        def func_deteriorarImagen_Grilla(self,nfranjasRowIn, nfranjasColIn, grosorFranjaIn, img_rawIM,img_rawIMGris, labelG):


            #img_rawIMGris, labelG, nfranjasRowIn, nfranjasColIn, grosorFranjaIn = args[0], args[1], args[2], args[3], args[4]
            #La gracia es que la imagen con grilla no se construya a cada rato.... con una por configuracion basta....

            nfranjasRow, nfranjasCol, grosorFranja = nfranjasRowIn, nfranjasColIn, grosorFranjaIn
            #nfranjasRowIn, nfranjasColIn, grosorFranjaIn = self.nfranjasRow, self.nfranjasCol, self.grosorFranja

            #img_rawIM=np.array(Image.open("Raw0.jpg"))
            #img_rawIM=np.array(Image.open("Raw0.jpg"))#imagen raw
            #img_rawIMGris = np.array(Image.open("Raw0.jpg").convert('L'))#imagen raw gris
            #labelG=np.array(Image.open("combinadasout0_2clases.png").convert('L'))
            #labelG[labelG == 1] = 255#Asi tendremos solo pixeles 0 y 255. Se normaliza entre 0 y 1 mas adelante


            nombreImagenFranjasRaw="2rawIMfranjas_"+self.prefijo+".png"
            nombreImagenFranjasRawGris="2rawIMGrisfranjas_"+self.prefijo+".png"
            nombreImagenFranjasLabeled="2labelGfranjas_"+self.prefijo+".png"                        
            #nombreImagenFranjasLabeledGT="2LabGTfranjas_"+self.prefijo+".png"  

            img_rawIM, img_rawIMGris, labelG= self.func_grillaNegra(nfranjasRow, nfranjasCol, grosorFranja, imagenRawIn=img_rawIM, imagenRawGrisIn=img_rawIMGris, imagenLabelIn=labelG)


            if((os.path.exists(self.directorioOutputs+nombreImagenFranjasRaw)==False) or 
               (os.path.exists(self.directorioOutputs+nombreImagenFranjasRawGris)==False) or
               (os.path.exists(self.directorioOutputs+nombreImagenFranjasLabeled)==False)):#guardar imagen con franjas una sola vez (raw)


                #nombreImagenFranjasRaw="borrameFranjasembrionRaw.jpg"

                #(self,imagenRawIn, imagenLabelIn, nfranjasRowIn, nfranjasColIn, grosorFranjaIn):      

                img_aux = Image.fromarray(img_rawIM)
                img_aux.save(self.directorioOutputs+nombreImagenFranjasRaw)

                img_aux = Image.fromarray(img_rawIMGris)
                img_aux.save(self.directorioOutputs+nombreImagenFranjasRawGris)

                #if(os.path.exists(r"outputsMasivos/"+nombreImagenFranjas)==False):#guardar imagen con franjas una sola vez (etiquetas)
                img_aux = Image.fromarray(labelG)
                img_aux.save(self.directorioOutputs+nombreImagenFranjasLabeled)

                #img_aux = Image.fromarray(labelG_GT)
                #img_aux.save(self.directorioOutputs+nombreImagenFranjasLabeledGT)

            return(img_rawIM, img_rawIMGris, labelG)


In [14]:
if False:
    class class_CRF_OutputDatoseImagen(class_CRF_OutputDatoseImagen):   

        #@classmethod
        #def func_grillaNegra(nfranjasRowIn, nfranjasColIn, grosorFranjaIn, imagenRawIn, imagenRawGrisIn, imagenLabelIn):      
        def func_grillaNegra(self, nfranjasRowIn, nfranjasColIn, grosorFranjaIn, *args,**kwargs):      
        #def func_grillaNegra(imagenRawIn, *args,**kwargs):      

            nfranjasRow, nfranjasCol, grosorFranja = nfranjasRowIn, nfranjasColIn, grosorFranjaIn
            #imagenRawIn, imagenRawGrisIn, imagenLabelIn = args[0], args[1]
            #imagenesIn=kwargs["imagenesIn"]

            #nfranjasRow=nfranjasRowIn
            #nfranjasCol=nfranjasColIn        
            #grosorFranja=grosorFranjaIn #igual para lineas verticales y horizontales

            imagenRawIn, imagenRawGrisIn, imagenLabelIn = kwargs.get("imagenRawIn"), kwargs.get("imagenRawGrisIn"), kwargs.get("imagenLabelIn")

            anchoImagen=imagenRawIn.shape[0]#podria haber sido imagenLabelIn, pero tienen las mismas dimensiones, cualquiera sirve
            altoImagen=imagenRawIn.shape[1]#podria haber sido imagenLabelIn, pero tienen las mismas dimensiones, cualquiera sirve
            #Numerototal=labelGAux.shape[0]
            #print(anchoImagen//(nfranjasCol+1))

            listaRow=list(range(1,altoImagen+1,altoImagen//(nfranjasRow+1)))#100+1 porque range no incluye el numero final
            listaCol=list(range(1,anchoImagen+1,anchoImagen//(nfranjasCol+1)))#100+1 porque range no incluye el numero final

            #print(listaCol)
            #500//3*3
            listaRow.remove(min(listaRow))#Siempre parte con el 1
            if(nfranjasRow<len(listaRow)):
                listaRow.remove(max(listaRow))

            listaCol.remove(min(listaCol))#Siempre parte con el 1
            if(nfranjasCol<len(listaCol)):
                listaCol.remove(max(listaCol))

            #K=0    
            K=255//2
            #K=255


            output=[]
            #if (imagenesIn==3):
            for i in listaRow:#Ojo que parte de 1 y las matrices parten del indice 0

                if(imagenRawIn is not None):
                    imagenRawIn[i-grosorFranja:i+grosorFranja,:,:]=K

                if(imagenRawGrisIn is not None):
                    imagenRawGrisIn[i-grosorFranja:i+grosorFranja,:]=K
                if(imagenLabelIn is not None):
                    imagenLabelIn[i-grosorFranja:i+grosorFranja,:]=K
                #imagenLabelGTIn[i-grosorFranja:i+grosorFranja,:]=K


            for i in listaCol:#Ojo que parte de 1 y las matrices parten del indice 0
                if(imagenRawIn is not None):
                    imagenRawIn[:,i-grosorFranja:i+grosorFranja,:]=K
                    output.append(imagenRawIn)
                if(imagenRawGrisIn is not None):    
                    imagenRawGrisIn[:,i-grosorFranja:i+grosorFranja]=K
                    output.append(imagenRawGrisIn)
                if(imagenLabelIn is not None):    
                    imagenLabelIn[:,i-grosorFranja:i+grosorFranja]=K   
                    output.append(imagenLabelIn)
                    #imagenLabelGTIn[:,i-grosorFranja:i+grosorFranja]=K

            #if (imagenesIn==1):
            #    for i in listaRow:#Ojo que parte de 1 y las matrices parten del indice 0
            #        imagenRawIn[i-grosorFranja:i+grosorFranja,:,:]=K

            #    for i in listaCol:#Ojo que parte de 1 y las matrices parten del indice 0
            #        imagenRawIn[:,i-grosorFranja:i+grosorFranja,:]=K

            return (output)
            #output=[]
            #for imagen_i in 
            #figure(figsize=(15,15)) 
            #plt.imshow(labelGAux)
            #if (imagenesIn==3):
            #    return(imagenRawIn, imagenRawGrisIn, imagenLabelIn)
            #if (imagenesIn==1):
            #    return(imagenRawIn)

In [15]:
if False:
    class class_CRF_OutputDatoseImagen(class_CRF_OutputDatoseImagen):   

        #def func_deteriorarImagen_Convolucion(self, k_convIn,img_rawIM,img_rawIMGris, labelG):
        def func_deteriorarImagen_Convolucion(self, k_convIn, *args,**kwargs):


            k_conv = k_convIn
            img_rawIM, img_rawIMGris, labelG = kwargs.get("img_rawIM"), kwargs.get("img_rawIMGris"), kwargs.get("labelG")
            lst_output=[]


            convolution_kernel = 1/(k_conv*k_conv)*np.ones((k_conv, k_conv))

            if (img_rawIM is not None):
                #ImagenRGB
                img_rawIM[:,:,0] = cv2.filter2D(img_rawIM[:,:,0], -1, convolution_kernel)#Esta es para calcular kernel bilateral
                img_rawIM[:,:,1] = cv2.filter2D(img_rawIM[:,:,1], -1, convolution_kernel)#Esta es para calcular kernel bilateral
                img_rawIM[:,:,2] = cv2.filter2D(img_rawIM[:,:,2], -1, convolution_kernel)#Esta es para calcular kernel bilateral

                nombreImagenRaw_conv="3(8)rawIMkconv_"+self.prefijo+".png" #8 entre parentesis porque es como la imagen "8ImagenRGBslots", del metodo original

                if(os.path.exists(self.directorioOutputs+nombreImagenRaw_conv)==False):#guardar imagen convolucionada una sola vez
                    img_aux2 = Image.fromarray(img_rawIM)
                    img_aux2.save(self.directorioOutputs+nombreImagenRaw_conv)

                lst_output.append(img_rawIM)

            if (img_rawIMGris is not None):
                #ImagenGris
                img_rawIMGris = cv2.filter2D(img_rawIMGris, -1, convolution_kernel)#Esta es para calcular kernel bilateral
                nombreImagenRawGris_conv="3rawIMGriskconv_"+self.prefijo+".png"

                if(os.path.exists(self.directorioOutputs+nombreImagenRawGris_conv)==False):#guardar imagen convolucionada una sola vez
                    img_aux2 = Image.fromarray(img_rawIMGris)
                    img_aux2.save(self.directorioOutputs+nombreImagenRawGris_conv)

                lst_output.append(img_rawIMGris)

            if (labelG is not None):
                #ImagenEtiquetada
                labelG = cv2.filter2D(labelG, -1, convolution_kernel)#Esta es para calcular probmap
                nombreImagenLabeled_conv="3labelGkconv_"+self.prefijo+".png"           

                if(os.path.exists(self.directorioOutputs+nombreImagenLabeled_conv)==False):#guardar imagen convolucionada una sola vez
                    img_aux2 = Image.fromarray(labelG)
                    img_aux2.save(self.directorioOutputs+nombreImagenLabeled_conv)

                lst_output.append(labelG)

            #if (imagenUsar=="usarEmbrion2Clases"):
            #Estas imagenes pueden estar o no convolucionadas (con k_conv=1 quedarian igual), y pueden o no tener franjas


            #nombreImagenRaw_conv=r"outputsMasivos/ImagenesTemporales/"+imagenUsar+"_RawRGBkconv"+str(k_conv)+"franjas"+str(nfranjasRowIn)+str(nfranjasColIn)+str(grosorFranjaIn)+".png"#las convolucionadas pueden tener o no franjas        
            #nombreImagenRawGris_conv=r"outputsMasivos/ImagenesTemporales/"+imagenUsar+"_RawGriskconv"+str(k_conv)+"franjas"+str(nfranjasRowIn)+str(nfranjasColIn)+str(grosorFranjaIn)+".png"#las convolucionadas pueden tener o no franjas        
            #nombreImagenLabeled_conv=r"outputsMasivos/ImagenesTemporales/"+imagenUsar+"_Labkconv"+str(k_conv)+"franjas"+str(nfranjasRowIn)+str(nfranjasColIn)+str(grosorFranjaIn)+".png"

            #self.filenameRaw_conv=filenameRaw_conv#para poder tirar esta linea self.imgdef=cv2.imread(self.filenameRaw)

            #imagenConvolucionada=imagenUsar+"Conv_Franjas.png"





            return(lst_output)
            #return(img_rawIM,img_rawIMGris, labelG)

In [16]:
class class_CRF_OutputDatoseImagen(class_CRF_OutputDatoseImagen):   
    
    def func_get_arrProbmap(self, labelG, probabilidades):#Toma una imagen y mientras mas cerca de 255 mas probabilidad de ser de la clase embrion
        
        imagenUsar, k_conv, nfranjasRowIn, nfranjasColIn, grosorFranjaIn = self.imagenUsar, self.k_conv, self.nfranjasRow, self.nfranjasCol, self.grosorFranja
        random.seed(10)
        
        #print("func_get_arrProbmap ANTES")
        #print(np.unique(labelG))
        
        #OJO QUE ESTO EXIGE QUE EL LABELG SEA ENTRE 0 Y 255
        labelG=(labelG - 0) / (255 - 0)#Se normaliza para hacer mas facil la asignacion de probabilidades
        
        #print("func_get_arrProbmap DESPUES")
        #print(np.unique(labelG))
        
        #listG=listG.tolist()
        listG=labelG.tolist()
        listGRows=(len(listG))
        listGCols=(len(listG[0]))
        
        p=0
        if (imagenUsar=="usarEmbrion2Clases" or imagenUsar=="usarMSRC"):
            p=p+1            
            #print(np.unique(labelG))
            #print("type listG: ",type(listG))
            #print("type listH: ",type(listH))
            #print("listG elemento 1: ",(listG[0][0] ))
            #print("listH elemento 1: ",(listH[0][0] ))
            #print("type listG elemento 1: ",type(listG[0][0] ))
            #print("type listH elemento 1: ",type(listH[0][0] ))

            int_CMax=probabilidades["int_CC"]#0.7
            int_CMin=probabilidades["int_CI"]#0.3
            int_IMax=probabilidades["int_II"]#0.55
            int_IMin=probabilidades["int_IC"]#0.45

            #int_CMax=0.51
            #int_CMin=0.49
            #int_IMax=0.51
            #int_IMin=0.49
            
            #Positivo=bordes, negativo=background
            #int_CC=0.7#Correcto clase correcta
            #int_CI=0.3
            #int_IC=0.45#Incorrecto(porque la mayor probabilidad en este caso la tiene una clase incorrecta) clase correcta 
            #int_II=0.55#incorrecto clase incorrecta 1
            
            labelGRows=labelG.shape[0]
            labelGCols=labelG.shape[1]


            for row in range(0,labelGRows):    
                for col in range(0,labelGCols):

                    #azar=random.choices(population=[0,1],weights=[0.45, 0.55],k=1)[0]
                    int_aux=listG[row][col]
                    #print(int_aux)#borrarprint
                    #print(labelG_Conv[row][col])
                    #if (azar==1):#Correcto. CLasifica "bien" en ambas clases con un 70% de probabilidades de hacerlo correctamente para cada clase

                    listG[row][col]=[int_CMax*(1-int_aux)+int_CMin*(int_aux), #Aca se le asigna una tupla a cada pixel
                                     int_CMax*(int_aux)+int_CMin*(1-int_aux)] #Es caso correcto. el primer termino va entre [int_CMin,int_CMax] y el segundo entre [int_CMin,int_CMax]. Si el pixel es 1 (vaca), la distribucion queda [0.3,0.7]. Tiene dimensiones igual a (alto,ancho,2)

                    #if (azar==0):#Incorrecto. CLasifica "mal" en ambas clases con un 55% de probabilidades de hacerlo incorrectamente para cada clase
                    #    listG[row][col]=[int_IMin*(1-int_aux)+int_IMax*(int_aux),
                    #                     int_IMin*(int_aux)+int_IMax*(1-int_aux)] #Es caso incorrecto. el primer termino va entre [int_IMin,int_IMax] y el segundo entre [int_IMin,int_IMax]. Si el pixel es 1 (vaca), la distribucion queda [0.7,0.3]

            arrProbmap = np.array(listG)
            #print("arrprobmap: "+str(arrProbmap.shape))
            #arrProbmap[:,:,:]=0.5
            
            mat_arrProbmap0 = np.matrix(arrProbmap[:,:,0])#El de la clase background
            mat_arrProbmap1 = np.matrix(arrProbmap[:,:,1])#El de la clase embrion

            #filenamearrProbmap0=r"outputsMasivos/ImagenesTemporales/"+imagenUsar+"arrprobmap0"+"_probmap_Kconv"+str(k_conv)+"franjas"+str(nfranjasRowIn)+str(nfranjasColIn)+"grosor"+str(grosorFranjaIn)+".txt"
            #filenamearrProbmap1=r"outputsMasivos/ImagenesTemporales/"+imagenUsar+"arrprobmap1"+"_probmap_Kconv"+str(k_conv)+"franjas"+str(nfranjasRowIn)+str(nfranjasColIn)+"grosor"+str(grosorFranjaIn)+".txt"
            
            nombreImagenarrProbmap0="4arrprobmap0"+self.prefijo+".txt"   
            nombreImagenarrProbmap1="4arrprobmap1"+self.prefijo+".txt"     
            
            if(os.path.exists(self.directorioOutputs+nombreImagenarrProbmap0)==False):
                #print("entro al txt de la matriz")
                with open(self.directorioOutputs+nombreImagenarrProbmap0,'wb') as f:
                    for line in mat_arrProbmap0:
                        np.savetxt(f, line, fmt='%.2f')
                        
            if(os.path.exists(self.directorioOutputs+nombreImagenarrProbmap1)==False):
                with open(self.directorioOutputs+nombreImagenarrProbmap1,'wb') as f:
                    for line in mat_arrProbmap1:
                        np.savetxt(f, line, fmt='%.2f')
            #input("Press a key to continue...")
                    
                    
        

        #filenameLabeled_probmap=r"outputsMasivos/ImagenesTemporales/"+self.imagenUsar+"_probmap_Kconv"+str(k_conv)+"franjas"+str(nfranjasRowIn)+str(nfranjasColIn)+"grosor"+str(grosorFranjaIn)+".png"
        
        #255 es probabilidad mas grande, osea que el fondo sea blanco para probmap0, esta bien porque es donde la clase fondo tiene mayor probabilidad
        nombreImagenLabeledprobmap0="4Probmap0"+self.prefijo+".png"
        nombreImagenLabeledprobmap1="4Probmap1"+self.prefijo+".png"
        if(os.path.exists(self.directorioOutputs+nombreImagenLabeledprobmap0)==False):#guardar imagen convolucionada una sola vez
            #imagenprobmap=np.zeros((labelG.shape[0], labelG.shape[1], 3))
            
            #imagenprobmap[:,:,0] = mat_arrProbmap0
            #imagenprobmap[:,:,1] = mat_arrProbmap1
            img_aux2 = Image.fromarray((mat_arrProbmap0 * 255/int_CMax).astype(np.uint8))#.astype(np.uint8)            
            img_aux2.save(self.directorioOutputs+nombreImagenLabeledprobmap0)
            
            
        if(os.path.exists(self.directorioOutputs+nombreImagenLabeledprobmap1)==False):
            img_aux2 = Image.fromarray((mat_arrProbmap1 * 255/int_CMax).astype(np.uint8))#.astype(np.uint8)            
            img_aux2.save(self.directorioOutputs+nombreImagenLabeledprobmap1)

        #Esto se va a caer si se usa otra imagen, porque depende del numero de clases y esta en el mismo flujo
        
        #print(np.unique(arrProbmap))
        #print(arrProbmap.min())
        #print(arrProbmap.max())
        if (p!=1):
            raise ValueError("No entró a ningun if/Entro a mas de un if, con la configuración para el probmap ingresada")  

        return(arrProbmap)

In [17]:
class class_CRF_OutputDatoseImagen(class_CRF_OutputDatoseImagen):   
    
    def func_get_arrProbmap10(self, labelG, probabilidades):
        probs = labelG.copy() #largoxanchoxrgb
        
        print("probs ",probs.shape)
        probs = (probs-probs.min()) / (probs.max()-probs.min())
        probs = 0.5 + 0.2 * (probs-0.5)

        # The first dimension needs to be equal to the number of classes.
        # Let's have one "foreground" and one "background" class.
        # So replicate the gaussian blob but invert it to create the probability
        # of the "background" class to be the opposite of "foreground".
        probs = np.tile(probs[np.newaxis,:,:],(2,1,1))

        probs[1,:,:] = 1 - probs[0,:,:]



        # Let's have a look:
        #plt.figure(figsize=(15,5))
        #plt.subplot(1,2,1); plt.imshow(probs[0,:,:]); plt.title('Foreground probability'); plt.axis('off'); plt.colorbar();
        #plt.subplot(1,2,2); plt.imshow(probs[1,:,:]); plt.title('Background probability'); plt.axis('off'); plt.colorbar();

        probs2=probs.copy()

        probs2[0,:,:]=probs[1,:,:]
        probs2[1,:,:]=probs[0,:,:]
        
        return(probs2)
        #U = unary_from_softmax(probs2)

In [18]:
class class_CRF_OutputDatoseImagen(class_CRF_OutputDatoseImagen):   
    
    def func_get_NameArrimageArraylabeledRowsColsDESUSO(self, imagenUsar, verbose):    
        #usarEmbrion=False
        #usarSelkam=False
        #usarPequeña=False
        #usarEmbrion2Clases=True
        #usarEmbrion3Clases=False
        #usarPatos=False

        
        #print("trabajando con arrays. Lo de aca me sirve como ground truth")
        #print("esta imagen se consiguió con GIMP ->image ->mode ->grayscale ->exportar")
        #labelG=np.array(Image.open(r"anno1-bGrey.png"))
        
        #print("ojo se esta transformando toda la informacion a grises entonces quizas se pierde informacion al pasar de rgb a gris")
        p=0
        if (imagenUsar=="usarEmbrion"):
            p=p+1
            nombreArchivo=r"combinadasout0.png"#segmentacion
            #nombreArchivo=r"combinadasout0_2clases.png"#segmentacion
            img_raw=r"Raw0.jpg"
            labelG=np.array(Image.open(nombreArchivo))
            img_rawIM=np.array(Image.open(img_raw))
            img_rawIMGris=img_rawIM
            #labelG[labelG == 113] = 0
            #labelG[labelG == 54] = 1

        if (imagenUsar=="usarEmbrion2Clases"):
            p=p+1
            #nombreArchivo=r"combinadasout0.png"#segmentacion
            nombreArchivo=r"combinadasout0_2clases.png"#segmentacion
            img_raw=r"Raw0.jpg"#imagen raw
            labelG=np.array(Image.open(nombreArchivo))
            img_rawIM=np.array(Image.open(img_raw))#imagen raw
            img_rawIMGris = np.array(Image.open(img_raw).convert('L'))#imagen raw gris
            
            #labelG[labelG == 113] = 0
            #labelG[labelG == 54] = 1

            if (verbose==True):
                print("img_rawIMGris: \n",img_rawIMGris.shape)
                print("np.unique(img_rawIMGris): \n",np.unique(img_rawIMGris))

                plt.title(img_raw+" .Esta es la imagen RAW convertida a un solo canal (grises), parece que es practicamente identica a la otra")
                plt.imshow(img_rawIMGris)
                #pred_mask_im=Image.fromarray(pred_mask) 
                plt.show()
                
        

        if (imagenUsar=="usarEmbrion3Clases"):
            p=p+1
            #nombreArchivo=r"combinadasout0.png"#segmentacion
            nombreArchivo=r"combinadasout0_3clases.png"#segmentacion
            img_raw=r"Raw0.jpg"
            labelG=np.array(Image.open(nombreArchivo))
            img_rawIM=np.array(Image.open(img_raw))
            img_rawIMGris=img_rawIM
            #labelG[labelG == 113] = 0
            #labelG[labelG == 54] = 1

        if (imagenUsar=="usarSelkam"):
            p=p+1
            nombreArchivo=r"selkam2.png"
            img_raw=r"selkam2.png"
            labelG=np.array(Image.open(nombreArchivo))
            img_rawIM=np.array(Image.open(img_raw))
            img_rawIMGris=img_rawIM
            #labelG[labelG == 113] = 0
            labelG[labelG > 0] = 1#En gimp me deja los bordes con una mezcla gris

        if (imagenUsar=="usarPequeña"):
            p=p+1
            nombreArchivo=r"PequeñaImagen10x10.png"
            img_raw=r"PequeñaImagen10x10.png"
            labelG=np.array(Image.open(nombreArchivo))
            img_rawIM=np.array(Image.open(img_raw))
            img_rawIMGris=img_rawIM
            #labelG[labelG == 113] = 0
            labelG[labelG > 0] = 1#En gimp me deja los bordes con una mezcla gris

        if (imagenUsar=="usarPatos"):
            p=p+1
            nombreArchivo=r"anno1-b.png"
            img_raw=r"im1.png"
            labelG=np.array(Image.open(nombreArchivo))
            img_rawIM=np.array(Image.open(img_raw))
            img_rawIMGris=img_rawIM
            #labelG[labelG == 113] = 0
            #labelG[labelG > 0] = 1#En gimp me deja los bordes con una mezcla gris
            
        if (imagenUsar=="usarPascal"):
            p=p+1
            nombreArchivo=r"2007_000187.png"#segmentacion
            img_raw=r"2007_000187.jpg"
            img_rawIM=np.array(Image.open(img_raw))
            labelG=np.array(Image.open(nombreArchivo))
            img_rawIMGris = np.array(Image.open(img_raw).convert('L'))
            labelG[labelG == 20] = 1
            labelG[labelG == 255] = 1
            
        if (imagenUsar=="usarMSRC"):
            p=p+1
            nombreArchivo=r"20220526_MRSCVaca.png"#segmentacion #ESTE ES PARA EL MIOU
            #nombreArchivo=r"vacasBorrosas.png"#segmentacion #ESTE ES PARA CALCULAR EL PROBMAP
            img_raw=r"1_13_s.bmp"#imagen original
            
            labelG=np.array(Image.open(nombreArchivo).convert('L'))#segmentacion
            img_rawIM=np.array(Image.open(img_raw))#imagen original
            
            img_rawIMGris = np.array(Image.open(img_raw).convert('L'))
            labelG[labelG>=1]=1#Este es para el MioU
            #img_rawIMGris=img_rawIM.copy()
        
        if (p!=1):
            raise ValueError("No entró a ningun/Entro a mas de un, if con la configuración ingresada")
        labelGRows, labelGCols = labelG.shape
        
        

        
        if(verbose==True):
            
            print("labelG.shape: \n",labelG.shape)
            print("np.unique(labelG): \n",np.unique(labelG))
            print("img_rawIM: \n",img_rawIM.shape)
            print("np.unique(img_rawIM): \n",np.unique(img_rawIM))
            
            plt.title(nombreArchivo+" .Esta es la imagen etiquetada. La imagen cruda (RAW) no es esta")
            plt.imshow(labelG)
            #pred_mask_im=Image.fromarray(pred_mask) 
            plt.show()


            plt.title(img_raw+" .Esta es la imagen RAW")
            plt.imshow(img_rawIM)
            #pred_mask_im=Image.fromarray(pred_mask) 
            plt.show()

        
        return(labelG, labelGRows, labelGCols, img_raw,img_rawIM, img_rawIMGris)

## Inicio crear probmap

In [19]:
if (False):
    imagenUsar="usarEmbrion2Clases"
    nombreImagenFranjas=imagenUsar+"Labeled_Franjas.png"
    if(os.path.exists(r"outputsMasivos/"+nombreImagenFranjas)==True):
        print("si")
        print(r"/outputsMasivosadsas/"+nombreImagenFranjas)

In [20]:
if False:
    class class_CRF_OutputDatoseImagenDESUSO(class_CRF_OutputDatoseImagen):   

        #def func_get_arrProbmap(self, imagenUsar, labelG, labelGRows, labelGCols, probabilidades, k_conv, franjaNegra):    
        def func_get_arrProbmap2(self, imagenUsar, labelG, labelGRows, labelGCols, probabilidades, k_conv, metodoFranjas, nfranjasRowIn, nfranjasColIn, grosorFranjaIn):   
            """No necesita nada mas que la imagen raw (o la version modificada de la imagen raw). No necesita sobel"""
            #usarEmbrion=False


            #usarEmbrion=True
            #print("Este tiene 5 clases")
            p=0
            if (imagenUsar=="usarEmbrion"):
                p=p+1
                import random
                random.seed(10)#MioU: 0.3675587
                #print("trabajando con listas")
                #print("tratando de construir probmap en esta celda")
                #print("si los valores erroneos aproposito bienen con mucha probabilidad, el CRF no se atreve a arreglarlos\
                #        por eso si es por ejemolo de [0,7,0.3] la probabilidad y esta mala, los granitos no los va a quitar el CRF")
                listG=labelG.tolist()
                listGRows=(len(listG))
                listGCols=(len(listG[0]))
                for row in range(0,labelGRows):    
                    for col in range(0,labelGCols):

                        azar=random.choices(population=[0,1],weights=[0.45, 0.55],k=1)[0]

                        if (azar==1):#Correcto
                            if (listG[row][col]==4):#Clase 4 (la mayor probabilidad esta en el puesto 4°)
                                listG[row][col]=[0.1,0.1,0.1,0.1,0.6]#La clase background que esta en el primer elemento tiene probabilidad 1
                            if (listG[row][col]==3):#Clase 3 (la mayor probabilidad esta en el puesto 3°)
                                listG[row][col]=[0.1,0.1,0.1,0.6,0.1]#La clase background que esta en el primer elemento tiene probabilidad 1
                            if (listG[row][col]==2):#Clase 2 (la mayor probabilidad esta en el puesto 2°)
                                listG[row][col]=[0.1,0.1,0.6,0.1,0.1]#La clase background que esta en el primer elemento tiene probabilidad 1
                            if (listG[row][col]==1):#Clase 1 (la mayor probabilidad esta en el puesto 1°)
                                listG[row][col]=[0.1,0.6,0.1,0.1,0.1]#La clase background que esta en el primer elemento tiene probabilidad 1
                            if (listG[row][col]==0):#Clase 0 (la mayor probabilidad esta en el puesto 0°)
                                listG[row][col]=[0.6,0.1,0.1,0.1,0.1]#La clase pato que esta en el segundo elemento tiene probabilidad 1
                        if (azar==0):#Incorrecto
                            if (listG[row][col]==4):
                                listG[row][col]=[0.0,0.0,0.0,0.55,0.45]#La clase background que esta en el primer elemento tiene probabilidad 1
                            if (listG[row][col]==3):
                                listG[row][col]=[0.0,0.0,0.0,0.45,0.55]#La clase background que esta en el primer elemento tiene probabilidad 1
                            if (listG[row][col]==2):
                                listG[row][col]=[0.0,0.55,0.45,0.0,0.0]#La clase background que esta en el primer elemento tiene probabilidad 1
                            if (listG[row][col]==1):
                                listG[row][col]=[0.55,0.45,0.0,0.0,0.0]#La clase background que esta en el primer elemento tiene probabilidad 1
                            if (listG[row][col]==0):
                                listG[row][col]=[0.45,0.55,0.0,0.0,0.0]#La clase pato que esta en el segundo elemento tiene probabilidad 1

                arrProbmap = np.array(listG)

            #usarEmbrion=True
            #print("Este tiene 3 clases")
            if (imagenUsar=="usarEmbrion3Clases"):
                p=p+1
                import random
                random.seed(10)#MioU: 0.3675587


                #print("trabajando con listas")
                #print("tratando de construir probmap en esta celda")
                #print("si los valores erroneos aproposito bienen con mucha probabilidad, el CRF no se atreve a arreglarlos\
                #        por eso si es por ejemolo de [0,7,0.3] la probabilidad y esta mala, los granitos no los va a quitar el CRF")
                listG=labelG.tolist()
                listGRows=(len(listG))
                listGCols=(len(listG[0]))


                int_CC=0.7#Correcto clase correcta
                int_CI=0.15#(1-int_AzarCClaseC)/2#dividido numero de clases

                int_IC=0.45#Incorrecto(porque la mayor probabilidad en este caso la tiene una clase incorrecta) clase correcta 
                int_II1=0.5#incorrecto clase incorrecta 1
                int_II2=0.05#incorrecto clase incorrecta 2

                for row in range(0,labelGRows):    
                    for col in range(0,labelGCols):

                        azar=random.choices(population=[0,1],weights=[0.45, 0.55],k=1)[0]

                        if (azar==1):#Correcto
                            #if (listG[row][col]==3):
                            #    listG[row][col]=[0.1,0.1,0.1,0.1,0.6]#La clase background que esta en el primer elemento tiene probabilidad 1
                            #if (listG[row][col]==3):
                            #    listG[row][col]=[0.1,0.1,0.1,0.6,0.1]#La clase background que esta en el primer elemento tiene probabilidad 1
                            if (listG[row][col]==2):
                                listG[row][col]=[int_CI,int_CI,int_CC]#La clase background que esta en el primer elemento tiene probabilidad 1
                            if (listG[row][col]==1):
                                listG[row][col]=[int_CI,int_CC,int_CI]#La clase background que esta en el primer elemento tiene probabilidad 1
                            if (listG[row][col]==0):
                                listG[row][col]=[int_CC,int_CI,int_CI]#La clase pato que esta en el segundo elemento tiene probabilidad 1
                        if (azar==0):#Incorrecto #Aca la clase se equivoca asi que la maxima probabilidad no puede estar en la posicion de la clase correspondiente (como en el caso azar correcto)
                            #if (listG[row][col]==3):
                            #    listG[row][col]=[0.0,0.0,0.0,0.55,0.45]#La clase background que esta en el primer elemento tiene probabilidad 1
                            #if (listG[row][col]==3):
                            #    listG[row][col]=[0.0,0.0,0.0,0.45,0.55]#La clase background que esta en el primer elemento tiene probabilidad 1
                            if (listG[row][col]==2):
                                listG[row][col]=[int_II2,int_II1,int_IC]#La clase background que esta en el primer elemento tiene probabilidad 1
                            if (listG[row][col]==1):
                                listG[row][col]=[int_II1,int_IC,int_II2]#La clase background que esta en el primer elemento tiene probabilidad 1
                            if (listG[row][col]==0):
                                listG[row][col]=[int_IC,int_II2,int_II1]#La clase pato que esta en el segundo elemento tiene probabilidad 1

                arrProbmap = np.array(listG)

            #usarSelkam=False
            #print("Estos tienen 2 clases")
            #if (imagenUsar=="usarSelkam" or imagenUsar=="usarPequeña" or imagenUsar=="usarEmbrion2Clases" or imagenUsar=="usarPatos" or imagenUsar=="usarPascal" or imagenUsar=="usarMSRC") : 
            #if (imagenUsar=="usarSelkam" or imagenUsar=="usarPequeña" or imagenUsar=="usarEmbrion2Clases" or imagenUsar=="usarPatos" or imagenUsar=="usarPascal") : 
            #if (imagenUsar=="usarSelkam" or imagenUsar=="usarPequeña" or imagenUsar=="usarEmbrion2Clases" or imagenUsar=="usarPatos"):
            if (imagenUsar=="usarSelkam" or imagenUsar=="usarPequeña" or imagenUsar=="usarPatos"):
                p=p+1
                import random
                random.seed(10)#MioU: 0.3675587
                #print("trabajando con listas")
                #print("tratando de construir probmap en esta celda")
                #print("si los valores erroneos aproposito bienen con mucha probabilidad, el CRF no se atreve a arreglarlos\
                #        por eso si es por ejemolo de [0,7,0.3] la probabilidad y esta mala, los granitos no los va a quitar el CRF")
                listG=labelG.tolist()
                listGRows=(len(listG))
                listGCols=(len(listG[0]))



                int_CC=probabilidades["int_CC"]
                int_CI=probabilidades["int_CI"]
                int_IC=probabilidades["int_IC"]
                int_II=probabilidades["int_II"]

                #Positivo=bordes, negativo=background
                #int_CC=0.7#Correcto clase correcta
                #int_CI=0.3
                #int_IC=0.45#Incorrecto(porque la mayor probabilidad en este caso la tiene una clase incorrecta) clase correcta 
                #int_II=0.55#incorrecto clase incorrecta 1



                for row in range(0,labelGRows):    
                    for col in range(0,labelGCols):

                        azar=random.choices(population=[0,1],weights=[0.45, 0.55],k=1)[0]

                        if (azar==1):#Correcto. CLasifica "bien" en ambas clases con un 70% de probabilidades de hacerlo correctamente para cada clase
                            if (listG[row][col]==1):
                                    listG[row][col]=[int_CI,int_CC]#La clase background que esta en el primer elemento tiene probabilidad 1
                            if (listG[row][col]==0):
                                listG[row][col]=[int_CC,int_CI]#La clase pato que esta en el segundo elemento tiene probabilidad 1
                        if (azar==0):#Incorrecto. CLasifica "mal" en ambas clases con un 55% de probabilidades de hacerlo incorrectamente para cada clase
                            if (listG[row][col]==1):
                                listG[row][col]=[0.55,0.45]#La clase background que esta en el primer elemento tiene probabilidad 1
                            if (listG[row][col]==0):
                                listG[row][col]=[0.45,0.55]#La clase pato que esta en el segundo elemento tiene probabilidad 1

                arrProbmap = np.array(listG)


            #if (imagenUsar=="usarMSRC" or imagenUsar=="usarPascal") : 
            if (imagenUsar=="usarMSRC" or imagenUsar=="usarPascal" or imagenUsar=="usarEmbrion2Clases") : 
                p=p+1


                #labelG_Conv = cv2.filter2D(labelG, -1, convolution_kernel)
                #labelG=labelG_Conv.copy()
                import random
                random.seed(10)#MioU: 0.3675587
                #print("trabajando con listas")
                #print("tratando de construir probmap en esta celda")
                #print("si los valores erroneos aproposito bienen con mucha probabilidad, el CRF no se atreve a arreglarlos\
                #        por eso si es por ejemolo de [0,7,0.3] la probabilidad y esta mala, los granitos no los va a quitar el CRF")



                if (True):
                    import cv2


                    #labelH = cv2.imread('20220526_MRSCVaca.png')
                    if (imagenUsar=="usarMSRC"):
                        labelG=np.array(Image.open("20220526_MRSCVaca.png").convert('L'))
                        labelG[labelG == 249] = 255
                        #k_conv=25#25
                    if (imagenUsar=="usarEmbrion2Clases"):
                        #img_rawIM=np.array(Image.open("Raw0.jpg"))
                        img_rawIMGris = np.array(Image.open("Raw0.jpg").convert('L'))#imagen raw gris

                        labelG=np.array(Image.open("combinadasout0_2clases.png").convert('L'))
                        labelG[labelG == 1] = 255#Asi tendremos solo pixeles 0 y 255. Se normaliza entre 0 y 1 mas adelante



                        if (metodoFranjas==True):
                            nombreImagenFranjasRaw="Embrion2C_Rawfranjas"+str(nfranjasRowIn)+str(nfranjasColIn)+str(grosorFranjaIn)+".png"
                            nombreImagenFranjasLabeled="Embrion2C_Labfranjas"+str(nfranjasRowIn)+str(nfranjasColIn)+str(grosorFranjaIn)+".png"                        

                            if((os.path.exists(r"outputsMasivos/ImagenesTemporales/"+nombreImagenFranjasRaw)==False) or (os.path.exists(r"outputsMasivos/ImagenesTemporales/"+nombreImagenFranjasLabeled)==False)):#guardar imagen con franjas una sola vez (raw)

                                #nombreImagenFranjasRaw="borrameFranjasembrionRaw.jpg"

                                #(self,imagenRawIn, imagenLabelIn, nfranjasRowIn, nfranjasColIn, grosorFranjaIn):      
                                #img_rawIMGris, labelG = self.func_grillaNegra(img_rawIMGris, labelG, nfranjasRowIn, nfranjasColIn, grosorFranjaIn)


                                #if False:
                                #    cantidadFranjas=1
                                #    if(franjaNegra!=0):
                                #        int_centro=labelG.shape[1]//(cantidadFranjas+1)
                                #        labelG[:,int_centro-franjaNegra:int_centro+franjaNegra]=0 #Franja vertical negra
                                #        img_rawIM[:,int_centro-franjaNegra:int_centro+franjaNegra]=0

                                #if(os.path.exists(r"outputsMasivos/"+nombreImagenFranjasRaw)==False):#guardar imagen con franjas una sola vez (raw)
                                img_aux = Image.fromarray(img_rawIMGris)
                                img_aux.save(r"outputsMasivos/ImagenesTemporales/"+nombreImagenFranjasRaw)

                                #if(os.path.exists(r"outputsMasivos/"+nombreImagenFranjas)==False):#guardar imagen con franjas una sola vez (etiquetas)
                                img_aux = Image.fromarray(labelG)
                                img_aux.save(r"outputsMasivos/ImagenesTemporales/"+nombreImagenFranjasLabeled)


                        #k_conv=10 es un buen desafío. Si k_conv=1 parece que la imagen queda igual
                    if (imagenUsar=="usarPascal"):
                        labelG=np.array(Image.open("2007_000187.png"))#.convert('L'))
                        labelG[labelG == 20] = 255#Asi tendremos solo pixeles 0 y 255. Se normaliza entre 0 y 1 mas adelante
                        #k_conv=25#25
                        #labelG[labelG == 255] = 1
                        #labelG=labelG.convert('L')



                    #Esta parte es comun a todas las imagenes de este if
                    #TOdas pasan por kconv... dejar en 1 si no se quiere convolucionar?
                    convolution_kernel = 1/(k_conv*k_conv)*np.ones((k_conv, k_conv))
                    img_rawIMGris = cv2.filter2D(img_rawIMGris, -1, convolution_kernel)#Esta es para calcular kernel bilateral
                    labelG = cv2.filter2D(labelG, -1, convolution_kernel)#Esta es para calcular probmap

                    #Estas imagenes pueden estar o no convolucionadas (con k_conv=1 quedarian igual), y pueden o no tener franjas
                    filenameRaw_conv=r"outputsMasivos/ImagenesTemporales/"+"Embrion2C_Rkconv"+str(k_conv)+"franjas"+str(nfranjasRowIn)+str(nfranjasColIn)+str(grosorFranjaIn)+".png"#las convolucionadas pueden tener o no franjas
                    self.filenameRaw_conv=filenameRaw_conv#para poder tirar esta linea self.imgdef=cv2.imread(self.filenameRaw)
                    filenameLabeled_conv=r"outputsMasivos/ImagenesTemporales/"+"Embrion2C_Lkconv"+str(k_conv)+"franjas"+str(nfranjasRowIn)+str(nfranjasColIn)+str(grosorFranjaIn)+".png"


                    #imagenConvolucionada=imagenUsar+"Conv_Franjas.png"
                    if(os.path.exists(filenameRaw_conv)==False):#guardar imagen convolucionada una sola vez
                            img_aux2 = Image.fromarray(img_rawIMGris)
                            img_aux2.save(filenameRaw_conv)


                    if(os.path.exists(filenameLabeled_conv)==False):#guardar imagen convolucionada una sola vez
                            img_aux2 = Image.fromarray(labelG)
                            img_aux2.save(filenameLabeled_conv)



                    labelG=(labelG - 0) / (255 - 0)#Se normaliza para hacer mas facil la asignacion de probabilidades


                    #listG=listG.tolist()



                listG=labelG.tolist()
                listGRows=(len(listG))
                listGCols=(len(listG[0]))

                #print(np.unique(labelG))
                #print("type listG: ",type(listG))
                #print("type listH: ",type(listH))
                #print("listG elemento 1: ",(listG[0][0] ))
                #print("listH elemento 1: ",(listH[0][0] ))
                #print("type listG elemento 1: ",type(listG[0][0] ))
                #print("type listH elemento 1: ",type(listH[0][0] ))

                int_CMax=probabilidades["int_CC"]#0.7
                int_CMin=probabilidades["int_CI"]#0.3
                int_IMax=probabilidades["int_II"]#0.55
                int_IMin=probabilidades["int_IC"]#0.45


                #Positivo=bordes, negativo=background
                #int_CC=0.7#Correcto clase correcta
                #int_CI=0.3
                #int_IC=0.45#Incorrecto(porque la mayor probabilidad en este caso la tiene una clase incorrecta) clase correcta 
                #int_II=0.55#incorrecto clase incorrecta 1



                for row in range(0,labelGRows):    
                    for col in range(0,labelGCols):

                        #azar=random.choices(population=[0,1],weights=[0.45, 0.55],k=1)[0]
                        int_aux=listG[row][col]
                        #print(int_aux)#borrarprint
                        #print(labelG_Conv[row][col])
                        #if (azar==1):#Correcto. CLasifica "bien" en ambas clases con un 70% de probabilidades de hacerlo correctamente para cada clase

                        listG[row][col]=[int_CMax*(1-int_aux)+int_CMin*(int_aux),
                                         int_CMax*(int_aux)+int_CMin*(1-int_aux)] #Es caso correcto. el primer termino va entre [int_CMin,int_CMax] y el segundo entre [int_CMin,int_CMax]. Si el pixel es 1 (vaca), la distribucion queda [0.3,0.7]. Tiene dimensiones igual a (alto,ancho,2)

                        #if (azar==0):#Incorrecto. CLasifica "mal" en ambas clases con un 55% de probabilidades de hacerlo incorrectamente para cada clase
                        #    listG[row][col]=[int_IMin*(1-int_aux)+int_IMax*(int_aux),
                        #                     int_IMin*(int_aux)+int_IMax*(1-int_aux)] #Es caso incorrecto. el primer termino va entre [int_IMin,int_IMax] y el segundo entre [int_IMin,int_IMax]. Si el pixel es 1 (vaca), la distribucion queda [0.7,0.3]


                arrProbmap = np.array(listG)

                filenameLabeled_probmap=r"outputsMasivos/ImagenesTemporales/"+self.imagenUsar+"_probmap_Kconv"+str(k_conv)+"franjas"+str(nfranjasRowIn)+str(nfranjasColIn)+"grosor"+str(grosorFranjaIn)+".png"
                if(os.path.exists(filenameLabeled_probmap)==False):#guardar imagen convolucionada una sola vez
                    img_aux2 = Image.fromarray((arrProbmap * 255).astype(np.uint8))
                    img_aux2.save(filenameLabeled_probmap)

                #Esto se va a caer si se usa otra imagen, porque depende del numero de clases y esta en el mismo flujo
                mat_arrProbmap0 = np.matrix(arrProbmap[:,:,0])
                mat_arrProbmap1 = np.matrix(arrProbmap[:,:,1])

                filenamearrProbmap0=r"outputsMasivos/ImagenesTemporales/"+self.imagenUsar+"arrprobmap0"+"_probmap_Kconv"+str(k_conv)+"franjas"+str(nfranjasRowIn)+str(nfranjasColIn)+"grosor"+str(grosorFranjaIn)+".txt"
                filenamearrProbmap1=r"outputsMasivos/ImagenesTemporales/"+self.imagenUsar+"arrprobmap1"+"_probmap_Kconv"+str(k_conv)+"franjas"+str(nfranjasRowIn)+str(nfranjasColIn)+"grosor"+str(grosorFranjaIn)+".txt"
                if(os.path.exists(filenamearrProbmap0)==False):
                    with open(filenamearrProbmap0,'wb') as f:
                        for line in mat_arrProbmap0:
                            np.savetxt(f, line, fmt='%.2f')
                if(os.path.exists(filenamearrProbmap1)==False):
                    with open(filenamearrProbmap1,'wb') as f:
                        for line in mat_arrProbmap1:
                            np.savetxt(f, line, fmt='%.2f')
                #input("Press a key to continue...")

            if (p!=1):
                raise ValueError("No entró a ningun if/Entro a mas de un if, con la configuración para el probmap ingresada")  

            return(arrProbmap, labelG, img_rawIMGris)

In [21]:
class class_CRF_OutputDatoseImagen(class_CRF_OutputDatoseImagen):   
    
    def func_plt_arrProbmap(self, arrProbmap1):    
        import tensorflow as tf
        arrProbmapPLT = tf.argmax(arrProbmap1, axis=-1)#Dimensiones (240, 320). Tipo tf.EagerTensor
        print(type(arrProbmapPLT))
        print(arrProbmapPLT.shape)
        arrProbmapPLT = arrProbmapPLT[..., tf.newaxis]#Dimensiones (240, 320, 1). Tipo tf.EagerTensor
        print(type(arrProbmapPLT))
        print(arrProbmapPLT.shape)
        arrProbmapPLT=np.squeeze(arrProbmapPLT[:,:,:]) 

        print("arrProbmapPLT IMAGEN: \n")

        plt.title("arrProbmapPLT: Maxima probabilidad Cada punto tiene probabilidad de ser 0(negro) y 1(blanco) y se grafica segun su maxima probabilidad")
        plt.imshow(arrProbmapPLT)
        plt.show()
        print("Se plotea la maxima probabilidad, se ve como una televisión de antena sin señal")

In [22]:
class class_CRF_OutputDatoseImagen(class_CRF_OutputDatoseImagen):   
    
    def func_get_sobels(self, img_rawIMGrisIn):   
        #import cv2
        #imgSobel=cv2.imread(nombreArchivo,0)
        #imgSobel=cv2.imread(img_raw,0)
        imgSobel=img_rawIMGrisIn

        sobelsdy = cv2.Sobel(imgSobel,cv2.CV_64F,1,0,ksize=3)
        #sobelsdy = np.absolute(sobelsdy)
        #sobelsdy = ((sobelsdy - np.min(sobelsdy)) / (np.max(sobelsdy) - np.min(sobelsdy)))*(255-0)+0
        #sobelsdy = sobelsdy.astype(int)
        #sobelsdy = sobelsdy.tolist()

        sobelsdx = cv2.Sobel(imgSobel,cv2.CV_64F,0,1,ksize=3)
        #sobelsdx = np.absolute(sobelsdx)
        #sobelsdx = ((sobelsdx - np.min(sobelsdx)) / (np.max(sobelsdx) - np.min(sobelsdx)))*(255-0)+0
        #sobelsdx = sobelsdx.astype(int)
        #sobelsdx = sobelsdx.tolist()

        #sobelsdx = np.absolute(sobelsdx)
        #sobelsdy = np.absolute(sobelsdy)

        sobelh=np.hypot(sobelsdx, sobelsdy)
        
        #filenameSobelx=r"outputsMasivos/ImagenesTemporales/"+self.imagenUsar+"_x"+".png"
        #filenameSobely=r"outputsMasivos/ImagenesTemporales/"+self.imagenUsar+"_y"+".png"
        #filenameSobelh=r"outputsMasivos/ImagenesTemporales/"+self.imagenUsar+"_h"+".png"
        
        #filenameSobelx=r"outputsMasivos/ImagenesTemporales/"+self.imagenUsar+"_Sobelx"+".png"
        #filenameSobely=r"outputsMasivos/ImagenesTemporales/"+self.imagenUsar+"_Sobely"+".png"
        #filenameSobelh=r"outputsMasivos/ImagenesTemporales/"+self.imagenUsar+"_Sobelh"+".png"
        
        #print("cambioaca")
        #Puede ser de 2 maneras: sacar sobelx y sobely en RGB y arctan pasarlo a gris o sacar sobelx y sobely en gris para tener arctan en gris.... Pero Arctan debe tener sí o sí un solo canal, porque en el otro estará la imagen RAW
        sobelsdx_im = Image.fromarray(sobelsdx).convert("L") #.convert('RGB')
        sobelsdy_im = Image.fromarray(sobelsdy).convert("L") #.convert('RGB')
        sobelsdh_im = Image.fromarray(sobelh).convert("L") #.convert('RGB')
        
        #sobelsdx_im = (sobelsdx)#.convert('RGB')
        #sobelsdy_im = (sobelsdy)#.convert('RGB')
        #sobelsdh_im = (sobelh)#.convert('RGB')
           
           
        nombreImagenSobelx="5Sobelx_"+self.prefijo+".png"
        nombreImagenSobely="5Sobely_"+self.prefijo+".png"
        nombreImagenSobelh="5Sobelh_"+self.prefijo+".png"
        

        
        if(os.path.exists(self.directorioOutputs+nombreImagenSobelx)==False):
            #sobelsdx_im.save(self.directorioOutputs+nombreImagenSobelx)
            cv2.imwrite(self.directorioOutputs+nombreImagenSobelx, np.float32(sobelsdx_im))
            
        if(os.path.exists(self.directorioOutputs+nombreImagenSobely)==False):
            #sobelsdy_im.save(self.directorioOutputs+nombreImagenSobely)
            cv2.imwrite(self.directorioOutputs+nombreImagenSobely, np.float32(sobelsdy_im))
            
        if(os.path.exists(self.directorioOutputs+nombreImagenSobelh)==False):
            #sobelsdh_im.save(self.directorioOutputs+nombreImagenSobelh)
            cv2.imwrite(self.directorioOutputs+nombreImagenSobelh, np.float32(sobelsdh_im))
        
        return(sobelsdx, sobelsdy, sobelh)

In [23]:
class class_CRF_OutputDatoseImagen(class_CRF_OutputDatoseImagen):   
    
    def func_plt_sobels(self, sobelsdx, sobelsdy, sobelh):   
        
        print("np.unique(sobelsdx): \n",np.unique(sobelsdx))
        print("sobelsdx.shape: \n",sobelsdx.shape)
        print("type(sobelsdx): \n",type(sobelsdx))
        plt.title("sobel x")
        plt.imshow(sobelsdx)
        plt.show()
        
        print("np.unique(sobelsdy): \n",np.unique(sobelsdy))
        print("sobelsdy.shape: \n",sobelsdy.shape)
        print("type(sobelsdy): \n",type(sobelsdy))
        plt.title("sobel y")
        plt.imshow(sobelsdy)
        plt.show()
        
        print("np.unique(sobelh): \n",np.unique(sobelh))
        print("sobelh.shape: \n",sobelh.shape)
        print("type(sobelh): \n",type(sobelh))
        plt.title("sobel hipotenusa")
        plt.imshow(sobelh)
        plt.show()


In [24]:
class class_CRF_OutputDatoseImagen(class_CRF_OutputDatoseImagen):   
    
    
    def func_get_Arctan(self, sobelsdx, sobelsdy): 
        imagenUsar=self.imagenUsar
        arr_arctan=np.arctan2(sobelsdy, sobelsdx)#Si se graba como png se truncan los decimales que van entre pi y -pi (o sea quedan solo los numeros -3,-2,-1,0,1,2,3)

        int_Normalizacion=255#Para dejarlo entre 0 (cuando vale -pi y 255 cuando vale pi) y 255
        #l=((l - np.min(l)) / (np.max(l) - np.min(l)))*int_Normalizacion
        arr_arctan=((arr_arctan - (-np.pi)) / (2*np.pi))*int_Normalizacion
        
        #arr_arctan_im = Image.fromarray(arr_arctan).convert('RGB')
        
        #filenameArrarctan=r"outputsMasivos/ImagenesTemporales/"+imagenUsar+"arctan_SinUmbral.jpg"
        
        
        nombreImagenArctan="6Arctan_SinUmbral_"+self.prefijo+".jpg"
        
        #arr_arctan=cv2.cvtColor(arr_arctan.astype(np.uint8), cv2.COLOR_BGR2GRAY)
        
        if((os.path.exists(self.directorioOutputs+nombreImagenArctan)==False)):
        #if(os.path.exists(filenameArrarctan)==False):
            #arr_arctan_im.save(self.directorioOutputs+nombreImagenArctan)
            cv2.imwrite(self.directorioOutputs+nombreImagenArctan, arr_arctan)
            
        return(arr_arctan)        

In [25]:
class class_CRF_OutputDatoseImagen(class_CRF_OutputDatoseImagen):   
    
    
    def func_filtrar_Umbral(self, arr_arctan, sobelh, umbralFiltro): 
        imagenUsar=self.imagenUsar
        #if (umbralFiltro!="SinUmbral"): #El filtro se hace para el arrarctan y para la imgraw
        a=sobelh>umbralFiltro#Y por que no directamente a arctan? Tira tues y falses....
        arr_arctan=arr_arctan*a             
        #img_rawIMGris=img_rawIMGris>umbralFiltro #La gris porque se necesita comprimir los 3 canales en uno solo. No se si tenga sentido hacer un umbral en la imagen raw. Ademas los umbrales no tienen porque ser lo missmo para matrices de arctangente y la imagen original. Que pasa si la imagen original es tenue y solo tiene pixeles con intensidad mayores que 100? Pero que sus arctan sean grandes y que si tenga sentido usar umbral 100? Por eso mejor dejar la imagen raw sin umbral


        arr_arctanfiltrado_im = Image.fromarray(arr_arctan).convert('RGB')        
        #filenameArrarctanfiltro=r"outputsMasivos/ImagenesTemporales/"+imagenUsar+"arctan_conUmbral"+str(umbralFiltro)+".jpg"            
        nombreImagenArctanUmbral="7Arctan_ConUmbral_"+self.prefijo+".jpg"
        #if(os.path.exists(filenameArrarctanfiltro)==False):
        if((os.path.exists(self.directorioOutputs+nombreImagenArctanUmbral)==False)):
            arr_arctanfiltrado_im.save(self.directorioOutputs+nombreImagenArctanUmbral)
        return(arr_arctan)


In [26]:
class class_CRF_OutputDatoseImagen(class_CRF_OutputDatoseImagen):   
    
    
    def func_construir_ImagenRGBslots(self, img_rawIM, img_rawIMGris, elegirRGBslots, arr_arctan=None):   
        
        #imagenUsar, k_conv, nfranjasRow, nfranjasCol, grosorFranja = self.imagenUsar, self.k_conv, self.nfranjasRow, self.nfranjasCol, self.grosorFranja 
        #self.rgbArray = np.zeros((arr_arctan.shape[0] ,arr_arctan.shape[1],3), 'uint8')#Hacer el esqueleto de la matriz para las imagenes
        rgbArray = np.zeros((img_rawIM.shape[0] ,img_rawIM.shape[1],3), 'uint8')#Hacer el esqueleto de la matriz para las imagenes
        #rgbArray = np.zeros((l.shape[0] ,l.shape[1],3),'float64')#intento fallido
        lst_configuraciones=["C1RC2GC3B", "C1RC2GC3B_2", "C1RawC2RawC3Raw", 
                             "C1RawC2SobelxC3Sobely", "C1RawC2SobelyC3Sobelx", "C1SobelxC2RawC3Sobely", "C1SobelxC2SobelyC3Raw", "C1SobelyC2SobelxC3Raw", "C1SobelyC2RawC3Sobelx",
                             
                             "C1RawC2RawC30", "C1RawC20C3Raw","C10C2RawC3Raw", 
                             "C1RawC2RawC3Angulo","C1RawC2AnguloC3Raw","C1AnguloC2RawC3Raw",
                             
                             "C1AnguloC2AnguloC30","C1AnguloC20C3Angulo","C10C2AnguloC3Angulo",
                             "C1AnguloC2AnguloC3Raw","C1AnguloC2RawC3Angulo","C1RawC2AnguloC3Angulo",
                             
                             "C1RawC20C30", "C10C2RawC30", "C10C20C3Raw",
                             "C1AnguloC20C30", "C10C2AnguloC30", "C10C20C3Angulo",
                             
                             
                               
                             ]
        
        
        
        
        import re
        #print("elegirRGBslots", elegirRGBslots)
        txt = elegirRGBslots
        C1 = re.search("(?<=C1)(.*)(?=C2)", txt)[0]
        C2 = re.search("(?<=C2)(.*)(?=C3)", txt)[0]
        C3 = re.search("(?<=C3)(.*)", txt)[0]

        p=0
        C1p, C2p, C3p = 0,0,0
        
        #Esta es la original parece
        if (elegirRGBslots=="C1RC2GC3B"):
            rgbArray=img_rawIM
            p=p+1
            
        #Esta es la original parece
        if (elegirRGBslots=="C1RC2GC3B_2"):
            rgbArray[..., 0] = img_rawIM[:, :, 0]
            rgbArray[..., 1] = img_rawIM[:, :, 1]
            rgbArray[..., 2] = img_rawIM[:, :, 2]
            p=p+1
            
        
        if (p==0):#0 o mas de 1
            #print("entro al if:"+"C1:"+str(C1)+"C2:"+str(C2)+"C3:"+str(C3))
            
        #############
        ###########################
        ##########################    
            if (C1=="Raw"):
                rgbArray[..., 0] = img_rawIMGris
                C1p=C1p+1
            if (C1=="Angulo"):
                rgbArray[..., 0] = arr_arctan        
                C1p=C1p+1
                #print("entro a c1 angulo")
            if (C1=="Sobelx"):
                rgbArray[..., 0] = self.sobelsdx
                C1p=C1p+1
            if (C1=="Sobely"):
                rgbArray[..., 0] = self.sobelsdy
                C1p=C1p+1
            if (C1=="0"):
                rgbArray[..., 0] = 0
                C1p=C1p+1


            if (C2=="Raw"):
                rgbArray[..., 1] = img_rawIMGris
                C2p=C2p+1
                #print("entro a c2 raw")
            if (C2=="Angulo"):
                rgbArray[..., 1] = arr_arctan 
                C2p=C2p+1
            if (C2=="Sobelx"):
                rgbArray[..., 1] = self.sobelsdx
                C2p=C2p+1
            if (C2=="Sobely"):
                rgbArray[..., 1] = self.sobelsdy
                C2p=C2p+1
            if (C2=="0"):
                rgbArray[..., 1] = 0
                C2p=C2p+1



            if (C3=="Raw"):
                rgbArray[..., 2] = img_rawIMGris
                C3p=C3p+1
            if (C3=="Angulo"):
                rgbArray[..., 2] = arr_arctan  
                C3p=C3p+1
            if (C3=="Sobelx"):
                rgbArray[..., 2] = self.sobelsdx
                C3p=C3p+1
            if (C3=="Sobely"):
                rgbArray[..., 2] = self.sobelsdy
                C3p=C3p+1
            if (C3=="0"):
                rgbArray[..., 2] = 0
                C3p=C3p+1
                #print("entro a c30")
        
        
        
        if False:
            plt.imshow(rgbArray)
            plt.title("rgbArray3Canales")
            plt.show()

            plt.imshow(rgbArray[:,:,0]) 
            plt.title("rgbArray[:,:,0]")
            plt.show()

            plt.imshow(rgbArray[:,:,1]) 
            plt.title("rgbArray[:,:,1]")
            plt.show()

            plt.imshow(rgbArray[:,:,2]) 
            plt.title("rgbArray[:,:,2]")
            plt.show()
        
        
            
        
    
        
        if ((p==0 & C1p!=1 & C2p!=1 & C3p!=1) or (p>1)): #si no pasa por las config iniciales debe pasar una sola vez por c1,c2yc3. Y no puede pasar dos veces considerando las dos primeras configuraciones
        #p=1 no entra a este if. Antes no podia ser p=0 ahora si puede siempre que c1p,c2pyc3p sean 1   
        #c1p,c2pyc3p pueden ser mayores que 0 solo si p==0
            print("elegirRGBslots", elegirRGBslots)
            raise ValueError("No entró a ningun/Entro a mas de un, if con la configuración ingresada")
            
            #input("Press a key to continue...")
        
        #rgbArrayimg = Image.fromarray(rgbArray*255)#Se ve que es rojo
        rgbArrayimg = Image.fromarray(rgbArray)
        
        
        nombreImagenImagenRGBslots="8ImagenRGBslots_"+str(self.elegirRGBslots)+self.prefijo+".png"                
        #filenameRGBSlots=r"outputsMasivos/ImagenesTemporales/"+imagenUsar+elegirRGBslots+str(k_conv)+str(nfranjasRow)+str(nfranjasCol)+str(grosorFranja)+".png"
        if (os.path.exists(self.directorioOutputs+nombreImagenImagenRGBslots)==False): 
        #if(os.path.exists(filenameRGBSlots)==False):
            rgbArrayimg.save(self.directorioOutputs+nombreImagenImagenRGBslots)
            #rgbArrayimg.save(filenameRGBSlots)
        

        return(rgbArrayimg, rgbArray)

In [27]:
class class_CRF_OutputDatoseImagen(class_CRF_OutputDatoseImagen):   

    def func_influir_SobelEnSmothnessKernel(self, sobelsdx, sobelsdy, influirSobelEnSmothnessKernel):#El del color es appearence kernel   
        #print("FUnciona tanto para la libreria propia o la original")
        #print("No da buenos resultados cambiar que influya, por ultimo se saco esa conclusion")
        #print("Si es falso: deja todos los sobels en 0 y solo influye el ParametroXY")
        #print("Si es verdadero: considera todos los sobels y deja fijo el ParametroXY")
        if(influirSobelEnSmothnessKernel==False):#En este caso sobelsdx y sobelsdy no influyen (en el loop en src/denserf.cpp feature(0,j*W_+i) = i / (sxVic+anguloActualx); da solo sxVic (que es sx) ya que anguloActual aca seran convertidos a ceros)(lo mismo ocurre con sxy) y solo influye ParametroXY (ojo que no sea 0 o se indefine y se cae todo)
            sobelsdx=np.where(sobelsdx<=np.inf, 0, sobelsdx)
            #print("func_influir_SobelEnSmothnessKernel")
            #print("np.unique(sobelsdx): \n",np.unique(sobelsdx))
            #print("sobelsdx.shape: \n",sobelsdx.shape)
            #print("type(sobelsdx): \n",type(sobelsdx))

            sobelsdy=np.where(sobelsdy<=np.inf, 0, sobelsdy)
            #print("func_influir_SobelEnSmothnessKernel")
            #print("np.unique(sobelsdy): \n",np.unique(sobelsdy))
            #print("sobelsdy.shape: \n",sobelsdy.shape)
            #print("type(sobelsdy): \n",type(sobelsdy))
        if(influirSobelEnSmothnessKernel==True):#En este caso sí influyen sobelsdx y sobelsdy
            pass
            #print("nada")
            #ParametroXY=(1,1)#No se puede dejar en decimales... quizas hay una parte en el codigo src/densecrf.cpp en donde esta variable es del tipo int y no double o algo así. No se puede dejar en 0 porque en las partes de las imagenes con sobelx o sobely igual a 0, quedarian divisiones por cero
        
        
        filenameSobelpostx=r"outputsMasivos/ImagenesTemporales/"+self.imagenUsar+"_Sobelpostx"+".png"
        filenameSobelposty=r"outputsMasivos/ImagenesTemporales/"+self.imagenUsar+"_Sobelposty"+".png"
        
        
        sobelsdpostx_im = Image.fromarray(sobelsdx).convert("L") #.convert('RGB')
        sobelsdposty_im = Image.fromarray(sobelsdy).convert("L") #.convert('RGB')
        
        if(os.path.exists(filenameSobelpostx)==False):
            sobelsdpostx_im.save(filenameSobelpostx)
            
        if(os.path.exists(filenameSobelposty)==False):
            sobelsdposty_im.save(filenameSobelposty)
        
        return(sobelsdx, sobelsdy)

In [28]:
class class_CRF_OutputDatoseImagen(class_CRF_OutputDatoseImagen):   
    
    def func_aplicar_CRF_LibOriginal(self, arrProbmap, ParametroXY, ParametroRGB, imgdef, inferenceIn, pairwisebilateralCompat):   
        #import cv2

        #print("juega colo colo parametroRGB:", ParametroRGB)
        #ParametroXY=(1,1)
        #ParametroXY=ParametroXY1
        #ParametroRGB=(5,5,5)
        #ParametroRGB=ParametroRGB1

        #Metodo1="Nuevo" #Valores Nuevo, Original

        #if (False):#MODIFICAR esto para comparar metodo original con el propuesto
        #    if(MetodoRGBslots=="Original"):
                #Ojo esto asi como esta escrito tiene mayor jerarquia que el parametro "configuracion".. Si configuracion dice anguloangulo0, pero metodo es original, entonces se hace con el original no mas
                ###USAR parametro rgbim de addPairwiseBilateral para meter imagenes como esta pensado originalmente
                #print("aca va original")
         #       img = cv2.imread(img_raw) #Tiene dimensiones alto(imagen) x ancho(imagen) x 3(RGB) #Esto es lo mismo que usar el caso rawrawraw. O sea en vez de poner Metodo=rawrawraw, mejor pone en configuraciones [Raw,Raw,Raw]
                #plt.imshow(img_raw)
                
                
          #  if+(MetodoRGBslots=="Nuevo"):
                ###USAR parametro rgbim de addPairwiseBilateral para meter angulos que es una "propuesta nueva"
                #img = cv2.imread("angulos.png") #Tiene dimensiones alto(imagen) x ancho(imagen) x 3(RGB)   
           #     img=img_RGBasignacion
            
            #print("se podria corroborar dejar esto en False si rawrawraw da los mismos valores que Metodo==Original")
        
        img = imgdef
        #img=angulosCanal
        
        img = np.ascontiguousarray(img) #Tiene dimensiones alto x ancho x 3(RGB). Parece que es algo para acelerar el proceso. Ademas mantiene las dimensiones originales de img

        #img2=cv2.imread(r"combinadasout0.png") 
        #img2=cv2.imread(nombreArchivo) 
        
        probmap=arrProbmap.copy()#En vez de leer una imagen para el probmap yo se lo construyo
        if (self.probmapTipo=="probmapgeneradoOriginal" or self.probmapTipo=="probmapDesdeDCNN"):          

            probmap = np.transpose(probmap, (2, 0, 1))#Reacomodar las dimensiones
            probmap = probmap.copy(order='C') #Dimensiones: n_clases x alto(parametroFijado) x ancho(parametroFijado)

        #print("np.unique(img2)",np.unique(img2))
        #print(probmap.shape)
        #print(probmap.flags)

        n_classes, height, width = probmap.shape
        #print("n_classes",n_classes)

        #print("Este Codigo: unary = unary_from_softmax(probmap)")
        unary = unary_from_softmax(probmap) #Tiene dimensiones n_classes x (alto(parametroFijado)xancho(parametroFijado)). Una opción es hacer:-np.log(probs)
        #unary = np.ascontiguousarray(unary)    
        #print("unary", type(unary))    
        #print("unary",unary.shape)
        
        if (self.probmapTipo=="probmapgeneradoOriginal" or self.probmapTipo=="probmapDesdeDCNN"): 
            unary = unary.reshape((n_classes,-1)) #Tiene dimensiones n_classes x (alto(parametroFijado)xancho(parametroFijado))
        #print("unary", type(unary))    
        #print("unary",unary.shape)

        #print("Este Codigo: d = dcrf.DenseCRF2D(width, height, n_classes)")
        d = dcrf.DenseCRF2D(width, height, n_classes)# weight height labels
        #print("d", type(d))    
        #print("d",d.shape)

        #print("Este Codigo: d.setUnaryEnergy(unary)")
        d.setUnaryEnergy(unary) #d.setUnaryEnergy(-np.log(unary)) 
        #print("d", type(d))    
        #print("d",d.shape)
        
        
        #print("esto hay que cambiarlo para que sea el metodo original o el nuevo, especificamente se deben agregar los parametros angulo y angulo2 si se usa el metodo nuevo")
        #print("Este Codigo: d.addPairwiseGaussian(angulo=np.uint8(sobelsdx),angulo2=np.uint8(sobelsdy) ,sxy=(1,1), compat = 1)")
        #d.addPairwiseGaussian(angulo=np.uint8(sobelsdx),angulo2=np.uint8(sobelsdy) ,sxy=(1,1), compat = 1)#Este funciona con el original (20220214_lucasbeyer_pydensecrf_original)
        #d.addPairwiseGaussian(angulo=np.uint8([[0,1,2],[3,4,5]]),angulo2=np.uint8([[1,1],[1,1]]) ,sxy=(1,1), compat = 1)#Este funciona con el original (20220214_lucasbeyer_pydensecrf_original)
        #d.addPairwiseGaussian(sxy=(1,1), compat = 1)#Este funciona con el original (20220214_lucasbeyer_pydensecrf_original)
        #d.addPairwiseBilateral(angulo=np.uint8(sobelsdx),angulo2=np.uint8(sobelsdy),sxy=(1,1), srgb=5, rgbim=np.copy(img), compat=2)#Ojo que al 20220420 no le afecta los valores de sobelsdx y sobelsdy a addPairwiseBilateral (revisar codigo)
        #d.addPairwiseBilateral(angulo=np.uint8(sobelsdx),angulo2=np.uint8(sobelsdy),sxy=ParametroXY, srgb=ParametroRGB, rgbim=np.copy(img), compat=2)#Ojo que al 20220420 no le afecta los valores de sobelsdx y sobelsdy a addPairwiseBilateral (revisar codigo)
        #if(libreria=='20220506_lucasbeyer_pydensecrf_Original'):
            #for i in list(range(0,100)):
        #rgbArray = np.zeros((arr_arctan.shape[0] ,arr_arctan.shape[1],3), 'uint8')
        #d.addPairwiseBilateral(sxy=ParametroXY, srgb=ParametroRGB, rgbim=np.copy(self.rgbArray), compat=2)#Ojo que al 20220420 no le afecta los valores de sobelsdx y sobelsdy a addPairwiseBilateral (revisar codigo)
        #d.addPairwiseBilateral(sxy=ParametroXY, srgb=ParametroRGB, rgbim=np.copy(img), compat=2)#Ojo que al 20220420 no le afecta los valores de sobelsdx y sobelsdy a addPairwiseBilateral (revisar codigo)
        d.addPairwiseBilateral(sxy=ParametroXY, srgb=ParametroRGB, rgbim=np.copy(img), compat=pairwisebilateralCompat)#Ojo que al 20220420 no le afecta los valores de sobelsdx y sobelsdy a addPairwiseBilateral (revisar codigo)
        #d.addPairwiseBilateral(sxy=ParametroXY, srgb=ParametroRGB, rgbim=np.copy(self.borrame), compat=2)#Ojo que al 20220420 no le afecta los valores de sobelsdx y sobelsdy a addPairwiseBilateral (revisar codigo)
        #d.addPairwiseBilateral(sxy=ParametroXY, srgb=ParametroRGB, rgbim=np.copy(img_RGBasignacion), compat=2)#Ojo que al 20220420 no le afecta los valores de sobelsdx y sobelsdy a addPairwiseBilateral (revisar codigo)

        #print("Este Codigo: Q = d.inference(5)")
        Q = d.inference(inferenceIn) #Tipo: pydensecrf.eigen.MatrixXf

        #print("Este Codigo: Q = np.array(Q).reshape((n_classes, height, width))")
        Q = np.array(Q).reshape((n_classes, height, width)) #Dimensiones nclasses x alto x ancho
        #print(type(Q))
        #print(Q.shape)

        #print("Este Codigo: Q = np.transpose(Q, (1, 2, 0))")
        Q = np.transpose(Q, (1, 2, 0))
        #print("Si esta fuera de rango en el subindice en .cpp como que se inventa un numero y no tira error") 
        return(Q)

In [29]:
class class_CRF_OutputDatoseImagen(class_CRF_OutputDatoseImagen):   
    
    def func_aplicar_CRF_LibNueva(self, sobelsdx,sobelsdy, arrProbmap, ParametroXY, ParametroRGB, influirSobelEnSmothnessKernel, imgdef, inferenceIn, pairwisebilateralCompat):   
        #import cv2
        
        
        #sobelsdx, sobelsdy=self.func_influir_SobelEnSmothnessKernel(sobelsdx, sobelsdy, influirSobelEnSmothnessKernel)#Aca intervengo para cambiar valor de los sobels si es que no quiero que influyan en la libreria nueva al obtener self.img_RGBasignacion
        #print("juega colo colo parametroRGB:", ParametroRGB)
        #ParametroXY=(1,1)
        #ParametroXY=ParametroXY1
        #ParametroRGB=(5,5,5)
        #ParametroRGB=ParametroRGB1

        #Metodo1="Nuevo" #Valores Nuevo, Original

        #if (False):#MODIFICAR esto para comparar metodo original con el propuesto
        #    if(MetodoRGBslots=="Original"):
                #Ojo esto asi como esta escrito tiene mayor jerarquia que el parametro "configuracion".. Si configuracion dice anguloangulo0, pero metodo es original, entonces se hace con el original no mas
                ###USAR parametro rgbim de addPairwiseBilateral para meter imagenes como esta pensado originalmente
                #print("aca va original")
         #       img = cv2.imread(img_raw) #Tiene dimensiones alto(imagen) x ancho(imagen) x 3(RGB) #Esto es lo mismo que usar el caso rawrawraw. O sea en vez de poner Metodo=rawrawraw, mejor pone en configuraciones [Raw,Raw,Raw]
                #plt.imshow(img_raw)
                
                
          #  if(MetodoRGBslots=="Nuevo"):
                ###USAR parametro rgbim de addPairwiseBilateral para meter angulos que es una "propuesta nueva"
                #img = cv2.imread("angulos.png") #Tiene dimensiones alto(imagen) x ancho(imagen) x 3(RGB)   
           #     img=img_RGBasignacion
            
            #print("se podria corroborar dejar esto en False si rawrawraw da los mismos valores que Metodo==Original")
        
        img=imgdef
        #img=imgIn
        img = np.ascontiguousarray(img) #Tiene dimensiones alto x ancho x 3(RGB). Parece que es algo para acelerar el proceso. Ademas mantiene las dimensiones originales de img

        #img2=cv2.imread(r"combinadasout0.png") 
        #img2=cv2.imread(nombreArchivo) 
        img2=arrProbmap#En vez de leer una imagen para el probmap yo se lo construyo

        probmap = np.transpose(img2, (2, 0, 1))#Reacomodar las dimensiones
        probmap = probmap.copy(order='C') #Dimensiones: n_clases x alto(parametroFijado) x ancho(parametroFijado)

        #print("np.unique(img2)",np.unique(img2))
        #print(probmap.shape)
        #print(probmap.flags)

        n_classes, height, width = probmap.shape
        #print("n_classes",n_classes)

        #print("Este Codigo: unary = unary_from_softmax(probmap)")
        unary = unary_from_softmax(probmap) #Tiene dimensiones n_classes x (alto(parametroFijado)xancho(parametroFijado)). Una opción es hacer:-np.log(probs)
        #unary = np.ascontiguousarray(unary)    
        #print("unary", type(unary))    
        #print("unary",unary.shape)
        unary = unary.reshape((n_classes,-1)) #Tiene dimensiones n_classes x (alto(parametroFijado)xancho(parametroFijado))
        #print("unary", type(unary))    
        #print("unary",unary.shape)

        #print("Este Codigo: d = dcrf.DenseCRF2D(width, height, n_classes)")
        d = dcrf.DenseCRF2D(width, height, n_classes)# weight height labels
        #print("d", type(d))    
        #print("d",d.shape)

        #print("Este Codigo: d.setUnaryEnergy(unary)")
        d.setUnaryEnergy(unary) #d.setUnaryEnergy(-np.log(unary)) 
        #print("d", type(d))    
        #print("d",d.shape)
        
        
        #print("esto hay que cambiarlo para que sea el metodo original o el nuevo, especificamente se deben agregar los parametros angulo y angulo2 si se usa el metodo nuevo")
        #print("Este Codigo: d.addPairwiseGaussian(angulo=np.uint8(sobelsdx),angulo2=np.uint8(sobelsdy) ,sxy=(1,1), compat = 1)")
        #d.addPairwiseGaussian(angulo=np.uint8(sobelsdx),angulo2=np.uint8(sobelsdy) ,sxy=(1,1), compat = 1)#Este funciona con el original (20220214_lucasbeyer_pydensecrf_original)
        #d.addPairwiseGaussian(angulo=np.uint8([[0,1,2],[3,4,5]]),angulo2=np.uint8([[1,1],[1,1]]) ,sxy=(1,1), compat = 1)#Este funciona con el original (20220214_lucasbeyer_pydensecrf_original)
        #d.addPairwiseGaussian(sxy=(1,1), compat = 1)#Este funciona con el original (20220214_lucasbeyer_pydensecrf_original)
        #d.addPairwiseBilateral(angulo=np.uint8(sobelsdx),angulo2=np.uint8(sobelsdy),sxy=(1,1), srgb=5, rgbim=np.copy(img), compat=2)#Ojo que al 20220420 no le afecta los valores de sobelsdx y sobelsdy a addPairwiseBilateral (revisar codigo)
        #d.addPairwiseBilateral(angulo=np.uint8(sobelsdx),angulo2=np.uint8(sobelsdy),sxy=ParametroXY, srgb=ParametroRGB, rgbim=np.copy(img), compat=2)#Ojo que al 20220420 no le afecta los valores de sobelsdx y sobelsdy a addPairwiseBilateral (revisar codigo)
        #if(libreria=='20220506_lucasbeyer_pydensecrf_Original'):
            #for i in list(range(0,100)):
            #d.addPairwiseBilateral(sxy=ParametroXY, srgb=ParametroRGB, rgbim=np.copy(img), compat=2)#Ojo que al 20220420 no le afecta los valores de sobelsdx y sobelsdy a addPairwiseBilateral (revisar codigo)
            #d.addPairwiseBilateral(sxy=ParametroXY, srgb=ParametroRGB, rgbim=np.copy(img_RGBasignacion), compat=2)#Ojo que al 20220420 no le afecta los valores de sobelsdx y sobelsdy a addPairwiseBilateral (revisar codigo)
            
            
        #if(libreria=='20220211_lucasbeyer_pydensecrf'):
        d.addPairwiseBilateral(angulo=np.uint8(sobelsdx),angulo2=np.uint8(sobelsdy),sxy=ParametroXY, srgb=ParametroRGB, rgbim=np.copy(img), compat=pairwisebilateralCompat)
        #d.addPairwiseBilateral(angulo=np.uint8(sobelsdx),angulo2=np.uint8(sobelsdy),sxy=ParametroXY, srgb=ParametroRGB, rgbim=np.copy(img), compat=0)

        #print("Este Codigo: Q = d.inference(5)")
        Q = d.inference(inferenceIn) #Tipo: pydensecrf.eigen.MatrixXf

        #print("Este Codigo: Q = np.array(Q).reshape((n_classes, height, width))")
        Q = np.array(Q).reshape((n_classes, height, width)) #Dimensiones nclasses x alto x ancho
        #print(type(Q))
        #print(Q.shape)

        #print("Este Codigo: Q = np.transpose(Q, (1, 2, 0))")
        Q = np.transpose(Q, (1, 2, 0))
        #print("Si esta fuera de rango en el subindice en .cpp como que se inventa un numero y no tira error") 
        return(Q)

In [30]:
class class_CRF_OutputDatoseImagen(class_CRF_OutputDatoseImagen):   
    
    def func_modificar_outputCRF(self, Q):           
    
        #import tensorflow as tf
        Q = tf.argmax(Q, axis=-1)#Dimensiones (240, 320). Tipo tf.EagerTensor
        #print("type(Q): \n",type(Q))
        #print("Q.shape: \n",Q.shape)
        Q = Q[..., tf.newaxis]#Dimensiones (240, 320, 1). Tipo tf.EagerTensor
        #print("type(Q): \n",type(Q))
        #print("Q.shape: \n",Q.shape)
        Q=np.squeeze(Q[:,:,:]) 
        
        #filenamePredicted=r"outputsMasivos/ImagenesTemporales/predicted.png"
        nombreImagenPredicted="9Predicted_"+self.prefijo+"XY"+str(self.ParametroXY)+"RGB"+str(self.ParametroRGB)+".png"     
            
        if False:
            if(os.path.exists(self.directorioOutputs+nombreImagenPredicted)==False):

            #if(os.path.exists(filenamePredicted)==False):#guardar imagen convolucionada una sola vez
                img_aux2 = Image.fromarray((Q * 255).astype(np.uint8))
                img_aux2.save(self.directorioOutputs+nombreImagenPredicted)

        return(Q)

In [31]:
class class_CRF_OutputDatoseImagen(class_CRF_OutputDatoseImagen):   
    
    def func_get_mIoU(self, labelGIn, QIn, imagenUsar,num_classesIn):             
        #import tensorflow as tf
        #print("Despue de pasar por el CRF. Se ve que mejora respecto al resultado antes de entrar al CRF")
        #import tensorflow as tf
        #out1=(np.squeeze(np.array(Image.open(r"out1.png").convert('L'))))
        
        
        m = tf.keras.metrics.MeanIoU(num_classesIn)
       
            
        if(imagenUsar=="usarPascal"):
            labelGIn=labelGIn.flatten()
            QIn=QIn.flatten()
            
            #print("m.result().numpy(): \n",m.result().numpy())
            #print("Miou: m.result().numpy(): \n",miou)
        
        #print("qshape: ",Q.shape)
        #print("labelG: ",labelG.shape)
        #print("qunique: ",np.unique(Q))
        #print("labelGunique: ",np.unique(labelG))
        
        m.update_state(labelGIn, QIn)#Q es la salida del crf y labelG es el label original?
        #m.update_state(labelGIn, QIn)#Q es la salida del crf y labelG es el label original?
        miou=m.result().numpy()
        
        
        return(miou)

In [32]:
class class_CRF_OutputDatoseImagen(class_CRF_OutputDatoseImagen):   
    
    def func_plotearguardar_imagen_CRF(self, Q,miou,elegirRGBslots, umbralFiltro,ParametroXY,  ParametroRGB, MetodoRGBslots, probabilidades):           
        
        plt.title("Resultado de la inferencia con CRF \n"
                  +"miou:"+str(miou)+"\n"
                  +"elegirRGBslots:"+elegirRGBslots+"\n"
                  +"umbralFiltro:"+str(umbralFiltro)+"\n"
                  +"ParametroXY:"+str(ParametroXY)+"\n"
                  +"ParametroRGB:"+str(ParametroRGB)+"\n"                  
                  +"MetodoRGBslots:"+MetodoRGBslots+"\n"
                  +"probabilidades:"+str(probabilidades))
        
        plt.imshow(Q)
        plt.savefig(r"outputsMasivos/"+str(elegirRGBslots)+str(umbralFiltro)+str(ParametroXY)+str(ParametroRGB)+".png",bbox_inches='tight',facecolor="white")#bbox_inches='tight' es para que aparezca el titulo completo
        #pred_mask_im=Image.fromarray(pred_mask) 
        plt.show()
        
        
        return()

In [33]:
class class_CRF_OutputDatoseImagen(class_CRF_OutputDatoseImagen):   
    
    def func_write_ToCSV(self,miou, libreria, imagenUsar, elegirRGBslots, umbralFiltro, ParametroXY, ParametroRGB, MetodoRGBslots, probabilidades,influirSobelEnSmothnessKernel, 
                         nombrearchivoIn, metodoConv, k_conv, metodoFranjas, nfranjasRow, nfranjasCol, grosorFranja,
                        metodoErosionar, kernelErosionar, int_erosionIteraciones, inference, pairwisebilateralCompat):           
        
        from csv import DictWriter
        
        encabezado="miou,libreria,imagenUsar,elegirRGBslots,umbralFiltro,ParametroXY,ParametroRGB,MetodoRGBslots,probabilidades,influirSobelEnSmothnessKernel,metodoConv,k_conv,metodoFranjas,nfranjasRow,nfranjasCol,grosorFranja,metodoErosionar,kernelErosionar,int_erosionIteraciones,inference,pairwisebilateralCompat\n"
        encabezadoBulean=os.path.exists(r"outputsMasivos/"+nombrearchivoIn)#Para guardar el header una sola vez

        
        
        field_names = ['miou', 'libreria', 'imagenUsar','elegirRGBslots','umbralFiltro',
               'ParametroXY','ParametroRGB','MetodoRGBslots','probabilidades','influirSobelEnSmothnessKernel',
                      'metodoConv','k_conv', 'metodoFranjas', 'nfranjasRow', 'nfranjasCol', 'grosorFranja',
                      'metodoErosionar', 'kernelErosionar', 'int_erosionIteraciones', 'inference', 'pairwisebilateralCompat']
        
        # Dictionary
        dict={'miou':miou,'libreria':libreria,'imagenUsar':imagenUsar,'elegirRGBslots':elegirRGBslots,'umbralFiltro':umbralFiltro, 'ParametroXY':ParametroXY,
              'ParametroRGB':ParametroRGB,"MetodoRGBslots":MetodoRGBslots,'probabilidades':probabilidades,'influirSobelEnSmothnessKernel':str(influirSobelEnSmothnessKernel),
              'metodoConv':metodoConv,'k_conv':k_conv,'metodoFranjas':metodoFranjas,'nfranjasRow':nfranjasRow,'nfranjasCol':nfranjasCol,'grosorFranja':grosorFranja,
               'grosorFranja':grosorFranja, 'metodoErosionar':metodoErosionar, 'kernelErosionar':kernelErosionar, 'int_erosionIteraciones':int_erosionIteraciones, 'inference':inference,
              'pairwisebilateralCompat':pairwisebilateralCompat
             }
        
        #print(dict)
        #file_exists = os.path.isfile(r"outputsMasivos/"+nombrearchivoIn)
        #print(file_exists)
        
        with open(r"outputsMasivos/"+nombrearchivoIn, 'a') as f_object:
        
            if(encabezadoBulean==False):
                f_object.write(encabezado)
            
            dictwriter_object = DictWriter(f_object, fieldnames=field_names)
            
            
            #if not file_exists:
            #    print("hola")
            #er()
            
            
            dictwriter_object.writerow(dict)
            f_object.close()
            
            
    
        return()